In [1]:
import urllib.request, json
import requests
import sys
import os

savePath="../data/xeno-canto-dataset-full-all-Countries/"

## Performing Webscraping from xeno-canto API

In [2]:
def save_json(searchTerms, birdName, country):
    numPages = 1
    page = 1
    # create a path to save json files and recordings
    path = savePath + birdName.replace(':', '') + "/" + country
    if not os.path.exists(path):
        print("Creating subdirectory " + path + " for downloaded files...")
        os.makedirs(path)
        # download a json file for every page found in a query
    while page < numPages + 1:
        print("Loading page " + str(page) + "...")
        url = 'https://www.xeno-canto.org/api/2/recordings?query={0}&page={1}'.format(searchTerms.replace(' ', '%20'),
                                                                                      page)
        print(url)
        jsonPage = urllib.request.urlopen(url)
        jsondata = json.loads(jsonPage.read().decode('utf-8'))
        filename = path + "/jsondata_p" + str(page) + ".json"
        with open(filename, 'w') as outfile:
            json.dump(jsondata, outfile)
        # check number of pages
        numPages = jsondata['numPages']
        page = page + 1
    print("Found ", numPages, " pages in total.")
    # return number of files in json
    # each page contains 500 results, the last page can have less than 500 records
    print("Saved json for ", (numPages - 1) * 500 + len(jsondata['recordings']), " files")
    return path

# reads the json and return the list of values for selected json part
# i.e. "id" - ID number, "type": type of the bird sound such as call or song
# for all Xeno Canto files found with the given search terms.
def read_data(searchTerm, path):
    data = []
    numPages = 1
    page = 1
    # read all pages and save results in a list
    while page < numPages + 1:
        # read file
        with open(path + "/jsondata_p" + str(page) + ".json", 'r') as jsonfile:
            jsondata = jsonfile.read()
        jsondata = json.loads(jsondata)
        # check number of pages
        numPages = jsondata['numPages']
        # find "recordings" in a json and save a list with a search term
        for k in range(len(jsondata['recordings'])):
            data.append(jsondata["recordings"][k][searchTerm])
        page = page + 1
    return data

# downloads all sound files found with the search terms into xeno-canto directory
# into catalogue named after the search term (i.e. Apus apus)
# filename have two parts: the name of the bird in latin and ID number
def download(searchTerms, birdName, country):
    # create data/xeno-canto-dataset directory
    path = save_json(searchTerms, birdName, country)
    # get filenames: recording ID and bird name in latin from json
    filenamesID = read_data('id', path)
    filenamesCountry = read_data('cnt', path)
    # get website recording http download address from json
    fileaddress = read_data('file', path)
    numfiles = len(filenamesID)
    print("A total of ", numfiles, " files will be downloaded")
    basePath = "../data/xeno-canto-dataset-full-all-Countries/"
    for i in range(0, numfiles):
        print("Saving file ", i + 1, "/", numfiles,
              basePath + birdName.replace(':', '')+"/" +birdName.replace(':', '')+ filenamesID[
                  i] + ".mp3")
        file_path = basePath +birdName.replace(':', '') + "/"+ birdName.replace(':', '') + filenamesID[i] + ".mp3"
        #print(fileaddress[i])
        song = requests.get(fileaddress[i])
        with open(file_path,"wb") as f:
            f.write(song.content)

### List of all birds and countries from where sounds are recorded

In [3]:
birds = [
#  #'Dendrocopos major',
# # 'Motacilla flava',
##  'Rallus aquaticus',
#  'Gallinula chloropus',
#  'Corvus corax',
#  'Fulica atra',
#  'Acrocephalus palustris',
#  'Corvus cornix',
#  'Actitis hypoleucos',
#  'Troglodytes aedon',
#  'Aegithalos caudatus',
#  'Emberiza hortulana',
#  'Corvus corone',
#  'Numenius arquata',
#  'Ardea cinerea',
#  'Tachybaptus ruficollis',
#  'Muscicapa striata',
# #  'Acrocephalus dumetorum',
#  'Coturnix coturnix',
#  'Tringa totanus',
#  'Caprimulgus europaeus',
#  'Asio otus',
#  'Hippolais icterina',
#  'Tringa ochropus',
#  'Melospiza melodia',
#  'Tyto alba',
#  'Chroicocephalus ridibundus',
#  'Anthus pratensis',
#  'Haematopus ostralegus',
#  'Numenius phaeopus',
#  'Pluvialis apricaria',
#  'Luscinia svecica',
#  'Tringa nebularia',
#  'Nycticorax nycticorax',
#  'Anas platyrhynchos',
#  'Vanellus vanellus',
#  'Coloeus monedula',
#  'Charadrius dubius',
#  'Calidris alpina',
#  'Botaurus stellaris',
#  'Grus grus',
#  'Henicorhina leucophrys',
#  'Anser anser',
#  'Locustella naevia',
#  'Buteo buteo',
#  'Scolopax rusticola',
#  'Lophophanes cristatus',
#  'Charadrius hiaticula',
#  'Anas crecca',
#  'Ficedula parva',
#  'Tringa glareola',
#  'Dryobates minor',
#  'Turdus torquatus',
#  'Fringilla montifringilla',
#  'Hippolais polyglotta',
#  'Alcedo atthis',
#  'Motacilla cinerea',
#  'Phylloscopus inornatus',
#  'Zonotrichia capensis',
#  'Agelaius phoeniceus',
#  'Cygnus cygnus',
#  'Pipilo maculatus',
#  'Burhinus oedicnemus',
#  'Ixobrychus minutus',
#  'Carpodacus erythrinus',
#  'Thryomanes bewickii',
#  'Himantopus himantopus',
#  'Sterna hirundo',
#  'Porzana porzana',
#  'Cardinalis cardinalis',
#  'Saxicola rubicola',
#  'Apus apus',
#  'Cyclarhis gujanensis',
#  'Turdus migratorius',
#  'Anthus hodgsoni',
#  'Corvus frugilegus',
#  'Loxia pytyopsittacus',
#  'Crex crex',
#  'Phylloscopus ibericus',
 'Mareca penelope',
 'Phylloscopus trochiloides',
 'Glaucidium passerinum',
 'Picus canus',
 'Catharus ustulatus',
 'Lanius collurio',
 'Aegolius funereus',
 'Phylloscopus humei',
 'Toxostoma curvirostre',
 'Anthus spinoletta',
 'Dendrocoptes medius',
 'Accipiter gentilis',
 'Merops apiaster',
 'Otus scops',
 'Locustella luscinioides',
 'Phylloscopus fuscatus',
 'Phasianus colchicus',
 'Phylloscopus bonelli',
 'Thryothorus ludovicianus',
 'Pitangus sulphuratus',
 'Geothlypis trichas',
 'Pluvialis squatarola',
 'Melanitta nigra',
 'Copsychus saularis',
 'Junco hyemalis',
 'Larus argentatus',
 'Saxicola rubetra',
 'Acanthis flammea',
 'Vireo gilvus',
 'Myiarchus tyrannulus',
 'Henicorhina leucosticta',
 'Rupornis magnirostris',
 'Larus canus',
 'Falco tinnunculus',
 'Branta canadensis',
 'Coereba flaveola',
 'Setophaga aestiva',
 'Branta leucopsis',
 'Thalasseus sandvicensis',
 'Locustella fluviatilis',
 'Cygnus olor',
 'Tolmomyias sulphurescens',
 'Anser albifrons',
 'Anthus campestris',
 'Calcarius lapponicus',
 'Recurvirostra avosetta',
 'Egretta garzetta',
 'Ramphocaenus melanurus',
 'Limosa limosa',
 'Larus michahellis',
 'Glaucidium brasilianum',
 'Psittacula krameri',
 'Anthus cervinus',
 'Mimus polyglottos',
 'Nyctidromus albicollis',
 'Cistothorus palustris',
 'Zonotrichia leucophrys',
 'Zonotrichia albicollis',
 'Tyrannus melancholicus',
 'Mareca strepera',
 'Ficedula albicollis',
 'Phylloscopus borealis',
 'Oenanthe oenanthe',
 'Dumetella carolinensis',
 'Bubo virginianus',
 'Cyanocitta cristata']

#These are scientific names of birds

countries = [
    'United Kingdom',
    'France',
    'Spain',
    'Italy',
    'Germany',
    'Romania',
    'Croatia',
    'Belarus',
    'Iceland',
    'Greece',
    'Sweden',
    'Finland',
    'Portugal',
    'Croatia',
    'Ireland',
    'Belarus',
    'Poland',
    'Austria',
    'Scotland',
    'Switzerland',
    'India',
    'United States',
    'New Zealand',
    'Argentina',
    'South Africa',
    'Netherlands',
    'Australia',
    'Ecuador',
    'Brazil',
    'Mexico',
    'Ethiopia',
    'Bolivia',
    'Belgium',
    'Japan',
    'India',
    'South Korea',
    'Denmark',
    'Norway',
    'Russian Federation',
    'Canada',
    'Kenya',
    'Uganda',
    'Zambia',
    'Madagascar',
    'Morocco',
    'Cameroon',
    'Namibia',
    'Malawi',
    'Tanzania',
    'Ghana',
    'Ivory Coast',
    'Tunisia',
    'Gambia',
    'China',
    'Mongolia',
    'Indonesia',
    'Malaysia',
    'Thailand',
    'Philippines',
    'Taiwan',
    'Vietnam',
    'Laos',
    'Papua New Guinea',
    'Kazakhstan',
    'Singapore',
    'Iran',
    'Cambodia',
    'Sri Lanka',
    'Bhutan',
    'Myanmar',
    'Nepal',
    'Saudi Arabia',
    'Oman',
    'United Arab Emirates',
    'Israel',
    
]



# for bird in birds:
#     download(bird + ' type:song', bird.replace(' ', ''), 'countries')

In [4]:
dont_take = ['Picapica', 'Delichonurbicum', 'Turduspilaris',
'Coccothraustescoccothraustes', 'Garrulusglandarius', 'Motacillaalba',
'Passerdomesticus', 'Passermontanus', 'Emberizacitrinella',
'Columbapalumbus', 'Sittaeuropaea', 'Streptopeliadecaocto',
'Fringillacoelebs', 'Luscinialuscinia', 'Parusmajor',
'Sturnusvulgaris', 'Phoenicurusochruros', 'Cardueliscarduelis',
'Turdusphilomelos', 'Troglodytestroglodytes',
'Phoenicurusphoenicurus', 'Alaudaarvensis', 'Turdusmerula',
'Phylloscopustrochilus', 'Chlorischloris', 'Phylloscopuscollybita',
'Erithacusrubecula',
'Acrocephalusarundinaceus',
 'Acrocephalusschoenobaenus',
 'Acrocephalusscirpaceus',
 'Alaudaarvensis',
 'Anthustrivialis',
 'Athenenoctua',
 'Bubobubo',
 'Certhiabrachydactyla',
 'Certhiafamiliaris',
 'Cettiacetti',
 'Cisticolajuncidis',
 'Cuculuscanorus',
 'Currucacommunis',
 'Currucacurruca',
 'Currucamelanocephala',
 'Cyanistescaeruleus',
 'Dryocopusmartius',
 'Emberizacalandra',
 'Emberizacirlus',
 'Emberizaschoeniclus',
 'Ficedulahypoleuca',
 'Galeridacristata',
 'Gallinagogallinago',
 'Hirundorustica',
 'Jynxtorquilla',
 'Linariacannabina',
 'Loxiacurvirostra',
 'Lullulaarborea',
 'Lusciniamegarhynchos',
 'Melanocoryphacalandra',
 'Oriolusoriolus',
 'Parusmajor',
 'Periparusater',
 'Phylloscopussibilatrix',
 'Picusviridis',
 'Poecilemontanus',
 'Poecilepalustris',
 'Prunellamodularis',
 'Pycnonotussinensis',
 'Pyrrhulapyrrhula',
 'Regulusignicapilla',
 'Regulusregulus',
 'Serinusserinus',
 'Spinusspinus',
 'Strixaluco',
 'Sturnusunicolor',
 'Sylviaatricapilla',
 'Sylviaborin',
 'Turdusiliacus',
 'Turdusviscivorus',
 'Upupaepops']

In [5]:
len(dont_take)

78

In [6]:
len(birds)

66

In [7]:
# Final_birds_eu = []
# for i in birds:
#     if i not in dont_take:
#         Final_birds_eu.append(i)

In [8]:
# len(Final_birds_eu)

In [9]:
for bird in birds:
    for country in countries:
        try:
            download(bird + ' cnt:' + country + ' type:song', bird.replace(' ', ''), country)
        except:
            continue

Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Mareca%20penelope%20cnt:United%20Kingdom%20type:song&page=1
Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Mareca%20penelope%20cnt:France%20type:song&page=1
Found  1  pages in total.
Saved json for  10  files
A total of  10  files will be downloaded
Saving file  1 / 10 ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Marecapenelope872897.mp3
Saving file  2 / 10 ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Marecapenelope872896.mp3
Saving file  3 / 10 ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Marecapenelope780322.mp3
Saving file  4 / 10 ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Marecapenelope780288.mp3
Saving file  5 / 10 ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Marecapenelope695691.mp3
Saving file  6 / 10 ../data/xeno-canto-d

Found  1  pages in total.
Saved json for  2  files
A total of  2  files will be downloaded
Saving file  1 / 2 ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Marecapenelope350170.mp3
Saving file  2 / 2 ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Marecapenelope202752.mp3
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Mareca%20penelope%20cnt:Australia%20type:song&page=1
Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Mareca%20penelope%20cnt:Ecuador%20type:song&page=1
Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Mareca%20penelope%20cnt:Brazil%20type:song&page=1
Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=

Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Tunisia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Mareca%20penelope%20cnt:Tunisia%20type:song&page=1
Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/Gambia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Mareca%20penelope%20cnt:Gambia%20type:song&page=1
Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Marecapenelope/China for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Mareca%20penelope%20cnt:China%20type:song&page=1
Found  1  pages in total.
Saved

Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/France for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Phylloscopus%20trochiloides%20cnt:France%20type:song&page=1
Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Spain for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Phylloscopus%20trochiloides%20cnt:Spain%20type:song&page=1
Found  1  pages in total.
Saved json for  0  files
A total of  0  files will be downloaded
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Italy for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Phylloscopus%20trochiloides%20cnt:

Saving file  2 / 114 ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Phylloscopustrochiloides913920.mp3
Saving file  3 / 114 ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Phylloscopustrochiloides913912.mp3
Saving file  4 / 114 ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Phylloscopustrochiloides913445.mp3
Saving file  5 / 114 ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Phylloscopustrochiloides911697.mp3
Saving file  6 / 114 ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Phylloscopustrochiloides910943.mp3
Saving file  7 / 114 ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Phylloscopustrochiloides910243.mp3
Saving file  8 / 114 ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Phylloscopustrochiloides909887.mp3
Saving file  9 / 114 ../data/xeno-canto-dataset-full-all-Countries/Phylloscopustrochiloides/Phylloscopustrochil

...

Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Oenanthe%20oenanthe%20cnt:China%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Oenantheoenanthe/Mongolia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Oenanthe%20oenanthe%20cnt:Mongolia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Oenantheoenanthe/Indonesia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Oenanthe%20oenanthe%20cnt:Indonesia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Oenantheoenanthe/Malaysia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Oenanthe%20oenanthe%20cnt:Malaysia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Oenantheoenanthe/Thailand for downloaded files...
Loading page 1...
https://www.xeno-canto.

Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Dumetellacarolinensis/Brazil for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Dumetella%20carolinensis%20cnt:Brazil%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Dumetellacarolinensis/Mexico for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Dumetella%20carolinensis%20cnt:Mexico%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Dumetellacarolinensis/Ethiopia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Dumetella%20carolinensis%20cnt:Ethiopia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Dumetellacarolinensis/Bolivia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Dumetella%20carolinensis%20cnt:Bolivia%20type:song&page=1
Creating s

Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Bubovirginianus/Romania for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Bubo%20virginianus%20cnt:Romania%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Bubovirginianus/Croatia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Bubo%20virginianus%20cnt:Croatia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Bubovirginianus/Belarus for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Bubo%20virginianus%20cnt:Belarus%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Bubovirginianus/Iceland for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Bubo%20virginianus%20cnt:Iceland%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-al

Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Bubovirginianus/Mongolia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Bubo%20virginianus%20cnt:Mongolia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Bubovirginianus/Indonesia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Bubo%20virginianus%20cnt:Indonesia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Bubovirginianus/Malaysia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Bubo%20virginianus%20cnt:Malaysia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Bubovirginianus/Thailand for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Bubo%20virginianus%20cnt:Thailand%20type:song&page=1
Creating subdirectory ../data/xeno-canto-datas

Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Cyanocittacristata/Brazil for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Cyanocitta%20cristata%20cnt:Brazil%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Cyanocittacristata/Mexico for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Cyanocitta%20cristata%20cnt:Mexico%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Cyanocittacristata/Ethiopia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Cyanocitta%20cristata%20cnt:Ethiopia%20type:song&page=1
Creating subdirectory ../data/xeno-canto-dataset-full-all-Countries/Cyanocittacristata/Bolivia for downloaded files...
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Cyanocitta%20cristata%20cnt:Bolivia%20type:song&page=1
Creating subdirectory ../data/xeno

In [ ]:
import os  #Loriculusphilippensis


In [9]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is A0F3-AAC0

 Directory of C:\Users\anime\Downloads\Asian_Birds

12-09-2024  14:01    <DIR>          .
12-09-2024  14:01    <DIR>          ..
11-09-2024  19:17    <DIR>          .ipynb_checkpoints
11-09-2024  20:10    <DIR>          data
12-09-2024  14:01         3,878,515 Data_preparation.ipynb
               1 File(s)      3,878,515 bytes
               4 Dir(s)  614,596,923,392 bytes free


In [11]:
for i in os.listdir("../data/xeno-canto-dataset-full-all-Countries/"):
    print(len(os.listdir(os.path.join("../data/xeno-canto-dataset-full-all-Countries/",i))))

24
17
17
17
18
18
29
17
17
22
24
17
17
18
20
17
18
18
18
22
27
19
18
56
23
39
34
25
32
17
51
17
17
18
17
17
17
17
17
27
17
30
17
17
17
20
17
17
20
22
17
25
24
67
17
18
17
17
19
18
23
17
17
17
17
17
18
18
30
23
22
17
17
21
17
17
18
21
19
17
17
17
18
17
17
17
17
17
17
18
19
17
17
17
17
17
18
22
17
17
17
17
24
20
17
18
17
17
17
19
17
18
18
18
56
27
19
88
18
18
18
18
19
17
24
34
45
17
17
17
18
17
19
17
149
23
21
17
19
28
19
17
18
22
26
22
17
17
17
19
52
18
17
17
19
17
17
17
17
17
18
31
17
17
17
17
17
19
17
17
17
17
17
17
18
17
17
17
17
17
17
17
17
18
17
17
19
17
38
23
24
25
41
18
18
30
17
17
17
17
19
17
17
18
23
17
17
17
17
76
34
17
42
24
17
18
19
19
17
17
17
18
17
17
17
17
17
17
17
17
17
23
17
17
18
17
24
19
59
17
19
18
18
22
18
29
30
17
17
23
17
17
17
17
23
17
17
17
18
17
17
17
24
57
17
17
17
20
19
17
17
21
17
18
17
18
17
18
17
19
31
93
21
22
71
17
17
20
17
22
19
19
17
17
17
17
17
23
17
17
17
20
18
28
17
17
17
73
18
29
23
19
17
17
46
26
31
88
27
17
20
17
17
17
17
17
17
17
17
17
19
19
17


In [1]:
birds = [
    'Casuarius casuarius',
'Dendrocygna guttata',
'Branta hutchinsii',
'Anser rossii',
'Anser fabalis',
'Cygnus atratus',
'Sarkidiornis melanotos',
'Alopochen aegyptiaca',
'Radjah radjah',
'Tadorna ferruginea',
'Salvadorina waigiuensis',
'Asarcornis scutulata',
'Aix galericulata',
'Nettapus pulchellus',
'Sibirionetta formosa',
'Spatula clypeata',
'Mareca strepera',
'Anas luzonica',
'Marmaronetta angustirostris',
'Netta rufina',
'Aythya valisineria',
'Polysticta stelleri',
'Somateria fischeri',
'Histrionicus histrionicus',
'Melanitta perspicillata',
'Clangula hyemalis',
'Bucephala albeola',
'Mergellus albellus',
'Mergus merganser',
'Oxyura leucocephala',
'Aepypodius arfakianus',
'Talegalla cuvieri',
'Macrocephalon maleo',
'Eulipoa wallacei',
'Megapodius reinwardt',
'Caloperdix oculeus',
'Rollulus rouloul',
'Melanoperdix niger',
'Arborophila orientalis',
'Lerwa lerwa',
'Ithaginis cruentus',
'Tragopan melanocephalus',
'Tetraophasis szechenyii',
'Lophophorus impejanus',
'Pucrasia macrolopha',
'Tetrastes bonasia',
'Lagopus lagopus',
'Falcipennis falcipennis',
'Tetrao urogallus',
'Lyrurus tetrix',
'Rhizothera longirostris',
'Perdix hodgsoniae',
'Syrmaticus soemmerringii',
'Chrysolophus pictus',
'Phasianus colchicus',
'Catreus wallichii',
'Crossoptilon harmani',
'Lophura ignita',
'Rheinardia ocellata',
'Argusianus argus',
'Pavo cristatus',
'Pavo muticus',
'Tropicoperdix chloropus',
'Haematortyx sanguiniceps',
'Galloperdix spadicea',
'Polyplectron chalcurum',
'Bambusicola fytchii',
'Gallus lafayettii',
'Ortygornis pondicerianus',
'Ortygornis gularis',
'Francolinus pintadeanus',
'Tetraogallus himalayensis',
'Ammoperdix griseogularis',
'Synoicus ypsilophorus',
'Coturnix coromandelica',
'Alectoris melanocephala',
'Perdicula manipurensis',
'Eurostopodus archboldi',
'Lyncornis temminckii',
'Caprimulgus pulchellus',
'Podargus ocellatus',
'Batrachostomus cornutus',
'Aegotheles cristatus',
'Hemiprocne coronata',
'Collocalia troglodytes',
'Aerodramus unicolor',
'Mearnsia picina',
'Hirundapus celebensis',
'Cypsiurus parvus',
'Apus caffer',
'Otis tarda',
'Ardeotis arabs',
'Chlamydotis macqueenii',
'Houbaropsis bengalensis',
'Sypheotides indicus',
'Tetrax tetrax',
'Centropus andamanensis',
'Carpococcyx radiceus',
'Rhinortha chlorophaea',
'Taccocua leschenaultii',
'Zanclostomus javanicus',
'Rhamphococcyx calyorhynchus',
'Phaenicophaeus curvirostris',
'Dasylophus superciliosus',
'Clamator coromandus',
'Coccycua pumila',
'Eudynamys scolopaceus'
'Urodynamis taitensis',
'Scythrops novaehollandiae',
'Chrysococcyx minutillus',
'Cacomantis aeruginosus',
'Surniculus velutinus',
'Hierococcyx nisicolor',
'Cuculus poliocephalus',
'Syrrhaptes paradoxus',
'Pterocles alchata',
'Columba vitiensis',
'Streptopelia turtur',
'Macropygia unchall',
'Reinwardtoena reinwardti',
'Turacoena manadensis',
'Oena capensis',
'Chalcophaps indica',
'Henicophaps albifrons',
'Trugon terrestris',
'Geopelia striata',
'Caloenas nicobarica',
'Gallicolumba tristigmata',
'Pampusana hoedtii',
'Otidiphaps nobilis',
'Goura cristata',
'Phapitreron leucotis',
'Treron fulvicollis',
'Ptilinopus arcanus',
'Ducula poliocephala',
'Cryptophaps poecilorrhoa',
'Gymnophaps albertisii',
'Gymnophaps mada',
'Gymnophaps stalkeri',
'Heliopais personatus',
'Rallicula leucospila',
'Rallicula rubra',
'Rallicula forbesi',
'Rallicula mayri',
'Rallus aquaticus',
'Rallus indicus',
'Crex crex',
'Aramidopsis plateni',
'Lewinia striata',
'Lewinia mirifica',
'Lewinia pectoralis',
'Habroptila wallacii',
'Gallirallus calayanensis',
'Eulabeornis castaneoventris',
'Hypotaenidia okinawae',
'Hypotaenidia torquata',
'Hypotaenidia philippensis',
'Porzana carolina',
'Porzana porzana',
'Gallinula tenebrosa',
'Gallinula chloropus',
'Fulica atra',
'Porphyrio alleni',
'Porphyrio porphyrio',
'Coturnicops exquisitus',
'Zapornia parva',
'Zapornia tabuensis',
'Rallina eurizonoides',
'Gymnocrex plumbeiventris',
'Megacrex inepta',
'Poliolimnas cinereus',
'Gallicrex cinerea',
'Amaurornis phoenicurus',
'Amaurornis olivacea',
'Amaurornis magnirostris',
'Amaurornis isabellina',
'Amaurornis moluccana',
'Leucogeranus leucogeranus',
'Antigone canadensis',
'Antigone vipio',
'Antigone antigone',
'Antigone rubicunda',
'Grus virgo',
'Grus japonensis',
'Grus grus',
'Grus monacha',
'Grus nigricollis',
'Tachybaptus ruficollis',
'Tachybaptus tricolor',
'Tachybaptus novaehollandiae',
'Podiceps grisegena',
'Podiceps cristatus',
'Podiceps auritus',
'Podiceps nigricollis',
'Phoenicopterus roseus',
'Phoeniconaias minor',
'Turnix sylvaticus',
'Turnix maculosus',
'Turnix tanki',
'Turnix ocellatus',
'Turnix suscitator',
'Turnix worcesteri',
'Turnix everetti',
'Burhinus oedicnemus',
'Burhinus indicus',
'Burhinus capensis',
'Burhinus grallarius',
'Esacus recurvirostris',
'Esacus magnirostris',
'Haematopus bachmani',
'Haematopus ostralegus',
'Haematopus longirostris',
'Haematopus fuliginosus',
'Ibidorhyncha struthersii',
'Himantopus himantopus',
'Himantopus leucocephalus',
'Recurvirostra avosetta',
'Vanellus vanellus',
'Vanellus spinosus',
'Vanellus duvaucelii',
'Vanellus tectus',
'Vanellus malabaricus',
'Vanellus cinereus',
'Vanellus indicus',
'Vanellus miles',
'Vanellus gregarius',
'Vanellus leucurus',
'Erythrogonys cinctus',
'Pluvialis apricaria',
'Pluvialis fulva',
'Pluvialis dominica',
'Pluvialis squatarola',
'Charadrius hiaticula',
'Charadrius semipalmatus',
'Charadrius placidus',
'Charadrius dubius',
'Charadrius vociferus',
'Charadrius pecuarius',
'Charadrius alexandrinus',
'Charadrius nivosus',
'Charadrius javanicus',
'Charadrius ruficapillus',
'Charadrius peronii',
'Charadrius mongolus',
'Charadrius leschenaultii',
'Charadrius asiaticus',
'Charadrius veredus',
'Charadrius morinellus',
'Elseyornis melanops',
'Pluvianus aegyptius',
'Rostratula benghalensis',
'Irediparra gallinacea',
'Hydrophasianus chirurgus',
'Metopidius indicus',
'Bartramia longicauda',
'Numenius tahitiensis',
'Numenius phaeopus',
'Numenius minutus',
'Numenius borealis',
'Numenius madagascariensis',
'Numenius tenuirostris',
'Numenius arquata',
'Limosa lapponica',
'Limosa limosa',
'Limosa haemastica',
'Arenaria interpres',
'Arenaria melanocephala',
'Calidris tenuirostris',
'Calidris canutus',
'Calidris pugnax',
'Calidris falcinellus',
'Calidris acuminata',
'Calidris himantopus',
'Calidris ferruginea',
'Calidris temminckii',
'Calidris subminuta',
'Calidris pygmaea',
'Calidris ruficollis',
'Calidris alba',
'Calidris alpina',
'Calidris ptilocnemis',
'Calidris maritima',
'Calidris bairdii',
'Calidris minuta',
'Calidris minutilla',
'Calidris fuscicollis',
'Calidris subruficollis',
'Calidris melanotos',
'Calidris pusilla',
'Calidris mauri',
'Limnodromus semipalmatus',
'Limnodromus scolopaceus',
'Limnodromus griseus',
'Scolopax rusticola',
'Scolopax mira',
'Scolopax saturata',
'Scolopax rosenbergii',
'Scolopax bukidnonensis',
'Scolopax celebensis',
'Scolopax rochussenii',
'Lymnocryptes minimus',
'Gallinago solitaria',
'Gallinago hardwickii',
'Gallinago nemoricola',
'Gallinago stenura',
'Gallinago megala',
'Gallinago media',
'Gallinago gallinago',
'Xenus cinereus',
'Phalaropus tricolor',
'Phalaropus lobatus',
'Phalaropus fulicarius',
'Actitis hypoleucos',
'Actitis macularius',
'Tringa ochropus',
'Tringa solitaria',
'Tringa incana',
'Tringa brevipes',
'Tringa flavipes',
'Tringa totanus',
'Tringa stagnatilis',
'Tringa glareola',
'Tringa erythropus',
'Tringa nebularia',
'Tringa guttifer',
'Tringa melanoleuca',
'Dromas ardeola',
'Cursorius cursor',
'Cursorius coromandelicus',
'Rhinoptilus bitorquatus',
'Stiltia isabella',
'Glareola pratincola',
'Glareola maldivarum',
'Glareola nordmanni',
'Glareola lactea',
'Anous stolidus',
'Anous tenuirostris',
'Anous minutus',
'Anous ceruleus',
'Gygis alba',
'Rynchops flavirostris',
'Rynchops albicollis',
'Rissa tridactyla',
'Rissa brevirostris',
'Pagophila eburnea',
'Xema sabini',
'Chroicocephalus genei',
'Chroicocephalus philadelphia',
'Chroicocephalus novaehollandiae',
'Chroicocephalus brunnicephalus',
'Chroicocephalus ridibundus',
'Chroicocephalus cirrocephalus',
'Chroicocephalus saundersi',
'Hydrocoloeus minutus',
'Rhodostethia rosea',
'Leucophaeus atricilla',
'Leucophaeus pipixcan',
'Ichthyaetus relictus',
'Ichthyaetus audouinii',
'Ichthyaetus melanocephalus',
'Ichthyaetus ichthyaetus',
'Ichthyaetus leucophthalmus',
'Ichthyaetus hemprichii',
'Larus crassirostris',
'Larus canus',
'Larus delawarensis',
'Larus californicus',
'Larus marinus',
'Larus glaucescens',
'Larus hyperboreus',
'Larus glaucoides',
'Larus argentatus',
'Larus smithsonianus',
'Larus vegae',
'Larus cachinnans',
'Larus armenicus',
'Larus schistisagus',
'Larus fuscus',
'Gelochelidon nilotica',
'Hydroprogne caspia',
'Thalasseus bergii',
'Thalasseus bengalensis',
'Thalasseus bernsteini',
'Thalasseus sandvicensis',
'Sternula albifrons',
'Sternula saundersi',
'Sternula antillarum',
'Onychoprion aleuticus',
'Onychoprion lunatus',
'Onychoprion anaethetus',
'Onychoprion fuscatus',
'Sterna aurantia',
'Sterna dougallii',
'Sterna sumatrana',
'Sterna hirundo',
'Sterna repressa',
'Sterna paradisaea',
'Sterna acuticauda',
'Chlidonias hybrida',
'Chlidonias leucopterus',
'Chlidonias niger',
'Stercorarius maccormicki',
'Stercorarius antarcticus',
'Stercorarius skua',
'Stercorarius pomarinus',
'Stercorarius parasiticus',
'Stercorarius longicaudus',
'Alle alle',
'Uria lomvia',
'Uria aalge',
'Alca torda',
'Cepphus grylle',
'Cepphus columba',
'Cepphus carbo',
'Brachyramphus marmoratus',
'Brachyramphus perdix',
'Brachyramphus brevirostris',
'Synthliboramphus antiquus',
'Synthliboramphus wumizusume',
'Ptychoramphus aleuticus',
'Aethia psittacula',
'Aethia pusilla',
'Aethia pygmaea',
'Aethia cristatella',
'Cerorhinca monocerata',
'Fratercula arctica',
'Fratercula corniculata',
'Fratercula cirrhata',
'Phaethon aethereus',
'Phaethon rubricauda',
'Phaethon lepturus',
'Gavia stellata',
'Gavia arctica',
'Gavia pacifica',
'Gavia immer',
'Gavia adamsii',
'Oceanites oceanicus',
'Pelagodroma marina',
'Fregetta grallaria',
'Fregetta tropica',
'Phoebastria immutabilis',
'Phoebastria nigripes',
'Phoebastria albatrus',
'Diomedea exulans',
'Thalassarche cauta',
'Thalassarche carteri',
'Hydrobates pelagicus',
'Hydrobates furcatus',
'Hydrobates monorhis',
'Hydrobates matsudairae',
'Hydrobates leucorhous',
'Hydrobates castro',
'Hydrobates tristrami',
'Hydrobates microsoma',
'Macronectes giganteus',
'Fulmarus glacialis',
'Daption capense',
'Pachyptila desolata',
'Pachyptila belcheri',
'Pterodroma incerta',
'Pterodroma solandri',
'Pterodroma externa',
'Pterodroma neglecta',
'Pterodroma baraui',
'Pterodroma sandwichensis',
'Pterodroma inexpectata',
'Pterodroma cervicalis',
'Pterodroma nigripennis',
'Pterodroma hypoleuca',
'Pterodroma leucoptera',
'Pterodroma longirostris',
'Pseudobulweria aterrima',
'Pseudobulweria rostrata',
'Calonectris leucomelas',
'Calonectris borealis',
'Ardenna pacifica',
'Ardenna bulleri',
'Ardenna grisea',
'Ardenna tenuirostris',
'Ardenna carneipes',
'Ardenna gravis',
'Puffinus nativitatis',
'Puffinus puffinus',
'Puffinus yelkouan',
'Puffinus lherminieri',
'Puffinus persicus',
'Puffinus bailloni',
'Puffinus bannermani',
'Bulweria bulwerii',
'Bulweria fallax',
'Mycteria cinerea',
'Mycteria ibis',
'Mycteria leucocephala',
'Anastomus oscitans',
'Ciconia nigra',
'Ciconia abdimii',
'Ciconia episcopus',
'Ciconia stormi',
'Ciconia ciconia',
'Ciconia boyciana',
'Ephippiorhynchus asiaticus',
'Leptoptilos javanicus',
'Leptoptilos dubius',
'Leptoptilos crumenifer',
'Fregata andrewsi',
'Fregata minor',
'Fregata ariel',
'Morus bassanus',
'Papasula abbotti',
'Sula dactylatra',
'Sula sula',
'Sula leucogaster',
'Anhinga melanogaster',
'Anhinga novaehollandiae',
'Microcarbo pygmaeus',
'Microcarbo niger',
'Microcarbo melanoleucos',
'Urile urile',
'Urile pelagicus',
'Phalacrocorax nigrogularis',
'Phalacrocorax sulcirostris',
'Phalacrocorax fuscicollis',
'Phalacrocorax capillatus',
'Phalacrocorax carbo',
'Gulosus aristotelis',
'Nannopterum auritum',
'Threskiornis aethiopicus',
'Threskiornis melanocephalus',
'Threskiornis molucca',
'Threskiornis spinicollis',
'Pseudibis papillosa',
'Pseudibis davisoni',
'Pseudibis gigantea',
'Geronticus eremita',
'Nipponia nippon',
'Plegadis falcinellus',
'Platalea leucorodia',
'Platalea minor',
'Platalea alba',
'Platalea regia',
'Zonerodius heliosylus',
'Botaurus stellaris',
'Ixobrychus minutus',
'Ixobrychus dubius',
'Ixobrychus sinensis',
'Ixobrychus eurhythmus',
'Ixobrychus cinnamomeus',
'Ixobrychus flavicollis',
'Gorsachius magnificus',
'Gorsachius goisagi',
'Gorsachius melanolophus',
'Nycticorax nycticorax',
'Nycticorax caledonicus',
'Butorides striata',
'Ardeola ralloides',
'Ardeola grayii',
'Ardeola bacchus',
'Ardeola speciosa',
'Bubulcus ibis',
'Bubulcus coromandus',
'Ardea cinerea',
'Ardea pacifica',
'Ardea melanocephala',
'Ardea insignis',
'Ardea sumatrana',
'Ardea goliath',
'Ardea purpurea',
'Ardea alba',
'Egretta picata',
'Egretta novaehollandiae',
'Egretta garzetta',
'Egretta gularis',
'Egretta sacra',
'Egretta eulophotes',
'Scopus umbretta',
'Pelecanus onocrotalus',
'Pelecanus rufescens',
'Pelecanus philippensis',
'Pelecanus crispus',
'Pelecanus conspicillatus',
'Pandion haliaetus',
'Pandion cristatus',
'Elanus caeruleus',
'Chelictinia riocourii',
'Gypaetus barbatus',
'Neophron percnopterus',
'Pernis apivorus',
'Pernis ptilorhynchus',
'Pernis celebensis',
'Pernis steerei',
'Aviceda jerdoni',
'Aviceda subcristata',
'Aviceda leuphotes',
'Henicopernis longicauda',
'Gyps bengalensis',
'Gyps indicus',
'Gyps tenuirostris',
'Gyps rueppelli',
'Gyps himalayensis',
'Gyps fulvus',
'Sarcogyps calvus',
'Aegypius monachus',
'Torgos tracheliotos',
'Spilornis cheela',
'Spilornis klossi',
'Spilornis kinabaluensis',
'Spilornis rufipectus',
'Spilornis holospilus',
'Spilornis elgini',
'Pithecophaga jefferyi',
'Circaetus gallicus',
'Terathopius ecaudatus',
'Macheiramphus alcinus',
'Harpyopsis novaeguineae',
'Nisaetus cirrhatus',
'Nisaetus floris',
'Nisaetus nipalensis',
'Nisaetus kelaarti',
'Nisaetus alboniger',
'Nisaetus bartelsi',
'Nisaetus lanceolatus',
'Nisaetus philippensis',
'Nisaetus pinskeri',
'Nisaetus nanus',
'Lophotriorchis kienerii',
'Ictinaetus malaiensis',
'Clanga pomarina',
'Clanga hastata',
'Clanga clanga',
'Hieraaetus pennatus',
'Hieraaetus weiskei',
'Aquila rapax',
'Aquila nipalensis',
'Aquila heliaca',
'Aquila gurneyi',
'Aquila chrysaetos',
'Aquila audax',
'Aquila verreauxii',
'Aquila fasciata',
'Micronisus gabar',
'Melierax metabates',
'Erythrotriorchis buergersi',
'Megatriorchis doriae',
'Accipiter trivirgatus',
'Accipiter griseiceps',
'Accipiter badius',
'Accipiter butleri',
'Accipiter brevipes',
'Accipiter soloensis',
'Accipiter trinotatus',
'Accipiter novaehollandiae',
'Accipiter hiogaster',
'Accipiter fasciatus',
'Accipiter melanochlamys',
'Accipiter henicogrammus',
'Accipiter poliocephalus',
'Accipiter gularis',
'Accipiter virgatus',
'Accipiter nanus',
'Accipiter erythrauchen',
'Accipiter cirrocephalus',
'Accipiter rhodogaster',
'Accipiter nisus',
'Accipiter gentilis',
'Accipiter meyerianus',
'Circus aeruginosus',
'Circus spilonotus',
'Circus approximans',
'Circus assimilis',
'Circus cyaneus',
'Circus macrourus',
'Circus melanoleucos',
'Circus pygargus',
'Milvus milvus',
'Milvus migrans',
'Haliastur sphenurus',
'Haliastur indus',
'Haliaeetus leucogaster',
'Haliaeetus leucoryphus',
'Haliaeetus albicilla',
'Haliaeetus leucocephalus',
'Haliaeetus pelagicus',
'Haliaeetus humilis',
'Haliaeetus ichthyaetus',
'Butastur teesa',
'Butastur liventer',
'Butastur indicus',
'Buteo lagopus',
'Buteo hemilasius',
'Buteo japonicus',
'Buteo refectus',
'Buteo rufinus',
'Buteo buteo',
'Tyto tenebricosa',
'Tyto inexspectata',
'Tyto nigrobrunnea',
'Tyto sororcula',
'Tyto novaehollandiae',
'Tyto rosenbergii',
'Tyto alba',
'Tyto deroepstorffi',
'Tyto longimembris',
'Phodilus badius',
'Phodilus assimilis',
'Uroglaux dimorpha',
'Ninox rufa',
'Ninox connivens',
'Ninox rudolfi',
'Ninox boobook',
'Ninox novaeseelandiae',
'Ninox japonica',
'Ninox scutulata',
'Ninox obscura',
'Ninox randi',
'Ninox affinis',
'Ninox philippensis',
'Ninox sumbaensis',
'Ninox burhani',
'Ninox ochracea',
'Ninox ios',
'Ninox hypogramma',
'Ninox squamipila',
'Ninox forbesi',
'Ninox natalis',
'Ninox theomacha',
'Ninox punctulata',
'Taenioptynx brodiei',
'Aegolius funereus',
'Athene brama',
'Athene noctua',
'Athene blewitti',
'Surnia ulula',
'Glaucidium passerinum',
'Glaucidium cuculoides',
'Glaucidium castanopterum',
'Glaucidium radiatum',
'Glaucidium castanotum',
'Otus gurneyi',
'Otus sagittatus',
'Otus rufescens',
'Otus thilohoffmanni',
'Otus balli',
'Otus alfredi',
'Otus spilocephalus',
'Otus angelinae',
'Otus mirus',
'Otus longicornis',
'Otus mindorensis',
'Otus scops',
'Otus brucei',
'Otus sunia',
'Otus elegans',
'Otus magicus',
'Otus sulaensis',
'Otus beccarii',
'Otus manadensis',
'Otus siaoensis',
'Otus collari',
'Otus mantananensis',
'Otus alius',
'Otus umbra',
'Otus enganensis',
'Otus mentawi',
'Otus brookii',
'Otus bakkamoena',
'Otus lettia',
'Otus semitorques',
'Otus lempiji',
'Otus megalotis',
'Otus nigrorum',
'Otus everetti',
'Otus fuliginosus',
'Otus silvicola',
'Asio otus',
'Asio flammeus',
'Bubo scandiacus',
'Bubo bubo',
'Bubo bengalensis',
'Bubo ascalaphus',
'Bubo sumatranus',
'Bubo nipalensis',
'Bubo coromandus',
'Bubo philippensis',
'Bubo blakistoni',
'Ketupa zeylonensis',
'Ketupa flavipes',
'Ketupa ketupu',
'Strix seloputo',
'Strix ocellata',
'Strix leptogrammica',
'Strix aluco',
'Strix nivicolum',
'Strix butleri',
'Strix uralensis',
'Strix davidi',
'Strix nebulosa',
'Apalharpactes reinwardtii',
'Apalharpactes mackloti',
'Harpactes fasciatus',
'Harpactes kasumba',
'Harpactes diardii',
'Harpactes ardens',
'Harpactes whiteheadi',
'Harpactes orrhophaeus',
'Harpactes duvaucelii',
'Harpactes oreskios',
'Harpactes erythrocephalus',
'Harpactes wardi',
'Upupa epops',
'Lophoceros nasutus',
'Berenicornis comatus',
'Buceros rhinoceros',
'Buceros bicornis',
'Rhinoplax vigil',
'Anthracoceros marchei',
'Anthracoceros albirostris',
'Anthracoceros coronatus',
'Anthracoceros montani',
'Anthracoceros malayanus',
'Ocyceros griseus',
'Ocyceros gingalensis',
'Ocyceros birostris',
'Anorrhinus tickelli',
'Anorrhinus austeni',
'Anorrhinus galeritus',
'Aceros nipalensis',
'Rhyticeros plicatus',
'Rhyticeros narcondami',
'Rhyticeros undulatus',
'Rhyticeros everetti',
'Rhyticeros subruficollis',
'Rhyticeros cassidix',
'Rhabdotorrhinus waldeni',
'Rhabdotorrhinus leucocephalus',
'Rhabdotorrhinus exarhatus',
'Rhabdotorrhinus corrugatus',
'Penelopides manillae',
'Penelopides mindorensis',
'Penelopides affinis',
'Penelopides samarensis',
'Penelopides panini',
'Coracias naevius',
'Coracias benghalensis',
'Coracias temminckii',
'Coracias caudatus',
'Coracias abyssinicus',
'Coracias garrulus',
'Eurystomus glaucurus',
'Eurystomus orientalis',
'Eurystomus azureus',
'Actenoides monachus',
'Actenoides princeps',
'Actenoides lindsayi',
'Actenoides hombroni',
'Actenoides concretus',
'Melidora macrorrhina',
'Lacedo pulchella',
'Tanysiptera galatea',
'Tanysiptera ellioti',
'Tanysiptera riedelii',
'Tanysiptera carolinae',
'Tanysiptera hydrocharis',
'Tanysiptera nympha',
'Tanysiptera danae',
'Cittura cyanotis',
'Clytoceyx rex',
'Dacelo leachii',
'Dacelo tyro',
'Dacelo gaudichaud',
'Caridonax fulgidus',
'Pelargopsis capensis',
'Pelargopsis melanorhyncha',
'Pelargopsis amauroptera',
'Halcyon coromanda',
'Halcyon smyrnensis',
'Halcyon cyanoventris',
'Halcyon pileata',
'Halcyon leucocephala',
'Todiramphus nigrocyaneus',
'Todiramphus winchelli',
'Todiramphus diops',
'Todiramphus lazuli',
'Todiramphus macleayii',
'Todiramphus funebris',
'Todiramphus chloris',
'Todiramphus enigma',
'Todiramphus saurophagus',
'Todiramphus sanctus',
'Todiramphus australasia',
'Syma torotoro',
'Syma megarhyncha',
'Corythornis cristatus',
'Alcedo coerulescens',
'Alcedo euryzona',
'Alcedo meninting',
'Alcedo atthis',
'Alcedo hercules',
'Ceyx erithaca',
'Ceyx melanurus',
'Ceyx fallax',
'Ceyx lepidus',
'Ceyx cyanopectus',
'Ceyx argentatus',
'Ceyx azureus',
'Ceyx pusillus',
'Megaceryle lugubris',
'Ceryle rudis',
'Nyctyornis amictus',
'Nyctyornis athertoni',
'Meropogon forsteni',
'Merops albicollis',
'Merops orientalis',
'Merops persicus',
'Merops philippinus',
'Merops ornatus',
'Merops viridis',
'Merops leschenaulti',
'Merops apiaster',
'Psilopogon pyrolophus',
'Psilopogon virens',
'Psilopogon lagrandieri',
'Psilopogon zeylanicus',
'Psilopogon lineatus',
'Psilopogon viridis',
'Psilopogon faiostrictus',
'Psilopogon corvinus',
'Psilopogon chrysopogon',
'Psilopogon rafflesii',
'Psilopogon mystacophanos',
'Psilopogon javensis',
'Psilopogon flavifrons',
'Psilopogon franklinii',
'Psilopogon oorti',
'Psilopogon annamensis',
'Psilopogon faber',
'Psilopogon nuchalis',
'Psilopogon asiaticus',
'Psilopogon monticola',
'Psilopogon incognitus',
'Psilopogon henricii',
'Psilopogon armillaris',
'Psilopogon pulcherrimus',
'Psilopogon australis',
'Psilopogon eximius',
'Psilopogon rubricapillus',
'Psilopogon malabaricus',
'Psilopogon haemacephalus',
'Caloramphus fuliginosus',
'Indicator xanthonotus',
'Indicator archipelagicus',
'Jynx torquilla',
'Picumnus innominatus',
'Sasia abnormis',
'Sasia ochracea',
'Hemicircus concretus',
'Hemicircus canente',
'Yungipicus temminckii',
'Yungipicus canicapillus',
'Yungipicus ramsayi',
'Yungipicus kizuki',
'Picoides tridactylus',
'Dendrocoptes auriceps',
'Dendrocoptes medius',
'Dryobates minor',
'Dendrocopos hyperythrus',
'Dendrocopos macei',
'Dendrocopos analis',
'Dendrocopos atratus',
'Dendrocopos darjellensis',
'Dendrocopos himalayensis',
'Dendrocopos assimilis',
'Dendrocopos syriacus',
'Dendrocopos leucopterus',
'Dendrocopos major',
'Dendrocopos noguchii',
'Dendrocopos leucotos',
'Dryocopus javensis',
'Dryocopus hodgei',
'Dryocopus martius',
'Chrysophlegma miniaceum',
'Chrysophlegma mentale',
'Chrysophlegma flavinucha',
'Picus chlorolophus',
'Picus puniceus',
'Picus viridanus',
'Picus vittatus',
'Picus xanthopygaeus',
'Picus squamatus',
'Picus awokera',
'Picus viridis',
'Picus rabieri',
'Picus erythropygius',
'Picus canus',
'Dinopium shorii',
'Dinopium javanense',
'Dinopium benghalense',
'Chrysocolaptes lucidus',
'Chrysocolaptes haematribon',
'Chrysocolaptes xanthocephalus',
'Chrysocolaptes erythrocephalus',
]

In [14]:
All_birds = [
#     'Casuarius casuarius',
# 'Casuarius bennetti',
# 'Casuarius unappendiculatus',
# 'Dendrocygna guttata',
# 'Dendrocygna bicolor',
# 'Dendrocygna eytoni',
# 'Dendrocygna arcuata',
# 'Dendrocygna javanica',
# 'Branta bernicla',
# 'Branta ruficollis',
# 'Branta canadensis',
# 'Branta leucopsis',
# 'Branta hutchinsii',
# 'Anser indicus',
# 'Anser canagicus',
# 'Anser rossii',
# 'Anser caerulescens',
# 'Anser anser',
# 'Anser cygnoides',
# 'Anser fabalis',
# 'Anser brachyrhynchus',
# 'Anser serrirostris',
# 'Anser albifrons',
# 'Anser erythropus',
# 'Cygnus atratus',
# 'Cygnus olor',
# 'Cygnus buccinator',
# 'Cygnus columbianus',
# 'Cygnus cygnus',
# 'Sarkidiornis melanotos',
# 'Alopochen aegyptiaca',
# 'Radjah radjah',
# 'Tadorna tadorna',
# 'Tadorna ferruginea',
# 'Salvadorina waigiuensis',
# 'Asarcornis scutulata',
# 'Aix galericulata',
# 'Nettapus coromandelianus',
# 'Nettapus pulchellus',
# 'Sibirionetta formosa',
# 'Spatula querquedula',
# 'Spatula clypeata',
# 'Mareca strepera',
# 'Mareca falcata',
# 'Mareca penelope',
# 'Mareca americana',
# 'Anas superciliosa',
# 'Anas luzonica',
# 'Anas poecilorhyncha',
# 'Anas zonorhyncha',
# 'Anas platyrhynchos',
# 'Anas rubripes',
# 'Anas acuta',
# 'Anas crecca',
# 'Anas carolinensis',
# 'Anas gibberifrons',
# 'Anas albogularis',
# 'Anas gracilis',
# 'Marmaronetta angustirostris',
# 'Netta rufina',
# 'Aythya valisineria',
# 'Aythya americana',
# 'Aythya ferina',
# 'Aythya australis',
# 'Aythya baeri',
# 'Aythya nyroca',
# 'Aythya collaris',
# 'Aythya fuligula',
# 'Aythya marila',
# 'Aythya affinis',
# 'Polysticta stelleri',
# 'Somateria fischeri',
# 'Somateria spectabilis',
# 'Somateria mollissima',
# 'Histrionicus histrionicus',
# 'Melanitta perspicillata',
# 'Melanitta fusca',
# 'Melanitta deglandi',
# 'Melanitta nigra',
# 'Melanitta americana',
# 'Clangula hyemalis',
# 'Bucephala albeola',
# 'Bucephala clangula',
# 'Bucephala islandica',
# 'Mergellus albellus',
# 'Mergus merganser',
# 'Mergus serrator',
# 'Mergus squamatus',
# 'Oxyura leucocephala',
# 'Aepypodius arfakianus',
# 'Aepypodius bruijnii',
# 'Talegalla cuvieri',
# 'Talegalla fuscirostris',
# 'Talegalla jobiensis',
# 'Macrocephalon maleo',
# 'Eulipoa wallacei',
# 'Megapodius nicobariensis',
# 'Megapodius cumingii',
# 'Megapodius bernsteinii',
# 'Megapodius tenimberensis',
# 'Megapodius freycinet',
# 'Megapodius geelvinkianus',
# 'Megapodius decollatus',
# 'Megapodius reinwardt',
# 'Caloperdix oculeus',
# 'Rollulus rouloul',
# 'Melanoperdix niger',
# 'Arborophila torqueola',
# 'Arborophila rufipectus',
# 'Arborophila mandellii',
# 'Arborophila gingica',
# 'Arborophila rufogularis',
# 'Arborophila rubrirostris',
# 'Arborophila diversa',
# 'Arborophila cambodiana',
# 'Arborophila ardens',
# 'Arborophila crudigularis',
# 'Arborophila atrogularis',
# 'Arborophila brunneopectus',
# 'Arborophila davidi',
# 'Arborophila hyperythra',
# 'Arborophila campbelli',
# 'Arborophila rolli',
# 'Arborophila sumatrana',
# 'Arborophila javanica',
# 'Arborophila orientalis',
# 'Lerwa lerwa',
# 'Ithaginis cruentus',
# 'Tragopan melanocephalus',
# 'Tragopan satyra',
# 'Tragopan blythii',
# 'Tragopan temminckii',
# 'Tragopan caboti',
# 'Tetraophasis obscurus',
# 'Tetraophasis szechenyii',
# 'Lophophorus impejanus',
# 'Lophophorus sclateri',
# 'Lophophorus lhuysii',
# 'Pucrasia macrolopha',
# 'Tetrastes bonasia',
# 'Tetrastes sewerzowi',
# 'Lagopus lagopus',
# 'Lagopus muta',
# 'Falcipennis falcipennis',
# 'Tetrao urogallus',
# 'Tetrao urogalloides',
# 'Lyrurus tetrix',
# 'Lyrurus mlokosiewiczi',
# 'Rhizothera longirostris',
# 'Rhizothera dulitensis',
# 'Perdix hodgsoniae',
# 'Perdix perdix',
# 'Perdix dauurica',
# 'Syrmaticus soemmerringii',
# 'Syrmaticus reevesii',
# 'Syrmaticus mikado',
# 'Syrmaticus ellioti',
# 'Syrmaticus humiae',
# 'Chrysolophus pictus',
# 'Chrysolophus amherstiae',
# 'Phasianus colchicus',
# 'Phasianus versicolor',
# 'Catreus wallichii',
# 'Crossoptilon harmani',
# 'Crossoptilon crossoptilon',
# 'Crossoptilon mantchuricum',
# 'Crossoptilon auritum',
# 'Lophura edwardsi',
# 'Lophura swinhoii',
# 'Lophura bulweri',
# 'Lophura leucomelanos',
# 'Lophura nycthemera',
# 'Lophura erythrophthalma',
# 'Lophura diardi',
# 'Lophura inornata',
# 'Lophura ignita',
# 'Rheinardia ocellata',
# 'Argusianus argus',
# 'Pavo cristatus',
# 'Pavo muticus',
# 'Tropicoperdix chloropus',
# 'Tropicoperdix charltonii',
# 'Haematortyx sanguiniceps',
# 'Galloperdix spadicea',
# 'Galloperdix lunulata',
# 'Galloperdix bicalcarata',
# 'Polyplectron napoleonis',
# 'Polyplectron schleiermacheri',
# 'Polyplectron malacense',
# 'Polyplectron germaini',
# 'Polyplectron katsumatae',
# 'Polyplectron bicalcaratum',
# 'Polyplectron inopinatum',
# 'Polyplectron chalcurum',
# 'Bambusicola fytchii',
# 'Bambusicola thoracicus',
# 'Gallus varius',
# 'Gallus gallus',
# 'Gallus sonneratii',
# 'Gallus lafayettii',
# 'Ortygornis pondicerianus',
# 'Ortygornis gularis',
# 'Francolinus pintadeanus',
# 'Francolinus francolinus',
# 'Francolinus pictus',
# 'Tetraogallus tibetanus',
# 'Tetraogallus altaicus',
# 'Tetraogallus caucasicus',
# 'Tetraogallus caspius',
# 'Tetraogallus himalayensis',
# 'Ammoperdix griseogularis',
# 'Ammoperdix heyi',
# 'Synoicus ypsilophorus',
# 'Synoicus monorthonyx',
# 'Synoicus chinensis',
# 'Coturnix coturnix',
# 'Coturnix japonica',
# 'Coturnix coromandelica',
# 'Alectoris melanocephala',
# 'Alectoris chukar',
# 'Alectoris graeca',
# 'Alectoris philbyi',
# 'Alectoris magna',
# 'Perdicula asiatica',
# 'Perdicula argoondah',
# 'Perdicula erythrorhyncha',
# 'Perdicula manipurensis',
# 'Eurostopodus argus',
# 'Eurostopodus mystacalis',
# 'Eurostopodus diabolicus',
# 'Eurostopodus papuensis',
# 'Eurostopodus archboldi',
# 'Lyncornis temminckii',
# 'Lyncornis macrotis',
# 'Caprimulgus indicus',
# 'Caprimulgus jotaka',
# 'Caprimulgus europaeus',
# 'Caprimulgus aegyptius',
# 'Caprimulgus mahrattensis',
# 'Caprimulgus nubicus',
# 'Caprimulgus atripennis',
# 'Caprimulgus macrurus',
# 'Caprimulgus meesi',
# 'Caprimulgus andamanicus',
# 'Caprimulgus manillensis',
# 'Caprimulgus celebensis',
# 'Caprimulgus asiaticus',
# 'Caprimulgus affinis',
# 'Caprimulgus concretus',
# 'Caprimulgus pulchellus',
# 'Podargus ocellatus',
# 'Podargus papuensis',
# 'Batrachostomus auritus',
# 'Batrachostomus harterti',
# 'Batrachostomus septimus',
# 'Batrachostomus stellatus',
# 'Batrachostomus moniliger',
# 'Batrachostomus hodgsoni',
# 'Batrachostomus poliolophus',
# 'Batrachostomus mixtus',
# 'Batrachostomus javensis',
# 'Batrachostomus affinis',
# 'Batrachostomus chaseni',
# 'Batrachostomus cornutus',
# 'Aegotheles insignis',
# 'Aegotheles crinifrons',
# 'Aegotheles wallacii',
# 'Aegotheles albertisi',
# 'Aegotheles bennettii',
# 'Aegotheles affinis',
# 'Aegotheles cristatus',
# 'Hemiprocne coronata',
# 'Hemiprocne longipennis',
# 'Hemiprocne comata',
# 'Hemiprocne mystacea',
# 'Hydrochous gigas',
# 'Collocalia esculenta',
# 'Collocalia dodgei',
# 'Collocalia linchi',
# 'Collocalia troglodytes',
# 'Aerodramus unicolor',
# 'Aerodramus mearnsi',
# 'Aerodramus infuscatus',
# 'Aerodramus sororum',
# 'Aerodramus ceramensis',
# 'Aerodramus hirundinaceus',
# 'Aerodramus brevirostris',
# 'Aerodramus vulcanorum',
# 'Aerodramus whiteheadi',
# 'Aerodramus nuditarsus',
# 'Aerodramus salangana',
# 'Aerodramus vanikorensis',
# 'Aerodramus maximus',
# 'Aerodramus fuciphagus',
# 'Aerodramus germani',
# 'Aerodramus papuensis',
# 'Mearnsia picina',
# 'Mearnsia novaeguineae',
# 'Zoonavena sylvatica',
# 'Rhaphidura leucopygialis',
# 'Hirundapus caudacutus',
# 'Hirundapus cochinchinensis',
# 'Hirundapus giganteus',
# 'Hirundapus celebensis',
# 'Cypsiurus parvus',
# 'Cypsiurus balasiensis',
# 'Tachymarptis melba',
# 'Apus apus',
# 'Apus pallidus',
# 'Apus pacificus',
# 'Apus leuconyx',
# 'Apus cooki',
# 'Apus acuticauda',
# 'Apus affinis',
# 'Apus nipalensis',
# 'Apus caffer',
# 'Otis tarda',
# 'Ardeotis arabs',
# 'Ardeotis nigriceps',
# 'Ardeotis australis',
# 'Chlamydotis undulata',
# 'Chlamydotis macqueenii',
# 'Houbaropsis bengalensis',
# 'Sypheotides indicus',
# 'Tetrax tetrax',
# 'Centropus menbeki',
# 'Centropus chalybeus',
# 'Centropus unirufus',
# 'Centropus chlororhynchos',
# 'Centropus melanops',
# 'Centropus steerii',
# 'Centropus rectunguis',
# 'Centropus celebensis',
# 'Centropus superciliosus',
# 'Centropus nigrorufus',
# 'Centropus sinensis',
# 'Centropus toulou',
# 'Centropus goliath',
# 'Centropus viridis',
# 'Centropus bengalensis',
# 'Centropus bernsteini',
# 'Centropus phasianinus',
# 'Centropus andamanensis',
# 'Carpococcyx radiceus',
# 'Carpococcyx viridis',
# 'Carpococcyx renauldi',
# 'Rhinortha chlorophaea',
# 'Taccocua leschenaultii',
# 'Zanclostomus javanicus',
# 'Rhamphococcyx calyorhynchus',
# 'Phaenicophaeus curvirostris',
# 'Phaenicophaeus pyrrhocephalus',
# 'Phaenicophaeus sumatranus',
# 'Phaenicophaeus viridirostris',
# 'Phaenicophaeus diardi',
# 'Phaenicophaeus tristis',
# 'Dasylophus superciliosus',
# 'Dasylophus cumingi',
# 'Clamator coromandus',
# 'Clamator glandarius',
# 'Clamator jacobinus',
# 'Coccycua pumila',
# 'Eudynamys scolopaceus',
# 'Eudynamys melanorhynchus',
# 'Eudynamys orientalis',
# 'Urodynamis taitensis',
# 'Scythrops novaehollandiae',
# 'Chrysococcyx maculatus',
# 'Chrysococcyx xanthorhynchus',
# 'Chrysococcyx caprius',
# 'Chrysococcyx klaas',
# 'Chrysococcyx megarhynchus',
# 'Chrysococcyx basalis',
# 'Chrysococcyx osculans',
# 'Chrysococcyx ruficollis',
# 'Chrysococcyx lucidus',
# 'Chrysococcyx meyerii',
# 'Chrysococcyx minutillus',
# 'Cacomantis pallidus',
# 'Cacomantis leucolophus',
# 'Cacomantis castaneiventris',
# 'Cacomantis flabelliformis',
# 'Cacomantis sonneratii',
# 'Cacomantis merulinus',
# 'Cacomantis passerinus',
# 'Cacomantis variolosus',
# 'Cacomantis sepulcralis',
# 'Cacomantis aeruginosus',
# 'Surniculus velutinus',
# 'Surniculus lugubris',
# 'Surniculus dicruroides',
# 'Surniculus musschenbroeki',
# 'Hierococcyx vagans',
# 'Hierococcyx sparverioides',
# 'Hierococcyx bocki',
# 'Hierococcyx varius',
# 'Hierococcyx hyperythrus',
# 'Hierococcyx pectoralis',
# 'Hierococcyx fugax',
# 'Hierococcyx nisicolor',
# 'Cuculus poliocephalus',
# 'Cuculus crassirostris',
# 'Cuculus micropterus',
# 'Cuculus saturatus',
# 'Cuculus optatus',
# 'Cuculus lepidus',
# 'Cuculus canorus',
# 'Syrrhaptes tibetanus',
# 'Syrrhaptes paradoxus',
# 'Pterocles alchata',
# 'Pterocles exustus',
# 'Pterocles senegallus',
# 'Pterocles orientalis',
# 'Pterocles coronatus',
# 'Pterocles lichtensteinii',
# 'Pterocles indicus',
# 'Columba leuconota',
# 'Columba oenas',
# 'Columba eversmanni',
# 'Columba palumbus',
# 'Columba arquatrix',
# 'Columba hodgsonii',
# 'Columba pulchricollis',
# 'Columba elphinstonii',
# 'Columba torringtoniae',
# 'Columba punicea',
# 'Columba argentina',
# 'Columba palumboides',
# 'Columba janthina',
# 'Columba vitiensis',
# 'Streptopelia turtur',
# 'Streptopelia lugens',
# 'Streptopelia orientalis',
# 'Streptopelia bitorquata',
# 'Streptopelia decaocto',
# 'Streptopelia roseogrisea',
# 'Streptopelia semitorquata',
# 'Streptopelia tranquebarica',
# 'Spilopelia chinensis',
# 'Spilopelia senegalensis',
# 'Macropygia unchall',
# 'Macropygia amboinensis',
# 'Macropygia emiliana',
# 'Macropygia magna',
# 'Macropygia tenuirostris',
# 'Macropygia rufipennis',
# 'Macropygia nigrirostris',
# 'Macropygia mackinlayi',
# 'Macropygia ruficeps',
# 'Reinwardtoena reinwardti',
# 'Turacoena manadensis',
# 'Turacoena modesta',
# 'Oena capensis',
# 'Chalcophaps indica',
# 'Chalcophaps longirostris',
# 'Chalcophaps stephani',
# 'Henicophaps albifrons',
# 'Trugon terrestris',
# 'Geopelia striata',
# 'Geopelia placida',
# 'Geopelia maugeus',
# 'Geopelia humeralis',
# 'Caloenas nicobarica',
# 'Gallicolumba tristigmata',
# 'Gallicolumba rufigula',
# 'Gallicolumba luzonica',
# 'Gallicolumba crinigera',
# 'Gallicolumba platenae',
# 'Gallicolumba keayi',
# 'Gallicolumba menagei',
# 'Pampusana hoedtii',
# 'Otidiphaps nobilis',
# 'Goura cristata',
# 'Goura scheepmakeri',
# 'Goura victoria',
# 'Phapitreron leucotis',
# 'Phapitreron amethystinus',
# 'Phapitreron cinereiceps',
# 'Phapitreron brunneiceps',
# 'Treron fulvicollis',
# 'Treron olax',
# 'Treron vernans',
# 'Treron bicinctus',
# 'Treron pompadora',
# 'Treron affinis',
# 'Treron phayrei',
# 'Treron chloropterus',
# 'Treron axillaris',
# 'Treron aromaticus',
# 'Treron curvirostra',
# 'Treron griseicauda',
# 'Treron teysmannii',
# 'Treron floris',
# 'Treron psittaceus',
# 'Treron capellei',
# 'Treron phoenicopterus',
# 'Treron waalia',
# 'Treron apicauda',
# 'Treron oxyurus',
# 'Treron seimundi',
# 'Treron sphenurus',
# 'Treron sieboldii',
# 'Treron formosae',
# 'Ptilinopus cinctus',
# 'Ptilinopus dohertyi',
# 'Ptilinopus porphyreus',
# 'Ptilinopus marchei',
# 'Ptilinopus merrilli',
# 'Ptilinopus occipitalis',
# 'Ptilinopus fischeri',
# 'Ptilinopus jambu',
# 'Ptilinopus subgularis',
# 'Ptilinopus gularis',
# 'Ptilinopus mangoliensis',
# 'Ptilinopus leclancheri',
# 'Ptilinopus bernsteinii',
# 'Ptilinopus magnificus',
# 'Ptilinopus perlatus',
# 'Ptilinopus ornatus',
# 'Ptilinopus aurantiifrons',
# 'Ptilinopus wallacii',
# 'Ptilinopus superbus',
# 'Ptilinopus regina',
# 'Ptilinopus coronulatus',
# 'Ptilinopus pulchellus',
# 'Ptilinopus monacha',
# 'Ptilinopus rivoli',
# 'Ptilinopus solomonensis',
# 'Ptilinopus viridis',
# 'Ptilinopus iozonus',
# 'Ptilinopus hyogastrus',
# 'Ptilinopus granulifrons',
# 'Ptilinopus melanospilus',
# 'Ptilinopus nainus',
# 'Ptilinopus arcanus',
# 'Ducula poliocephala',
# 'Ducula forsteni',
# 'Ducula mindorensis',
# 'Ducula radiata',
# 'Ducula carola',
# 'Ducula aenea',
# 'Ducula nicobarica',
# 'Ducula perspicillata',
# 'Ducula neglecta',
# 'Ducula concinna',
# 'Ducula pacifica',
# 'Ducula myristicivora',
# 'Ducula rufigaster',
# 'Ducula basilica',
# 'Ducula chalconota',
# 'Ducula pistrinaria',
# 'Ducula rosacea',
# 'Ducula pickeringii',
# 'Ducula pinon',
# 'Ducula mullerii',
# 'Ducula zoeae',
# 'Ducula badia',
# 'Ducula lacernulata',
# 'Ducula cineracea',
# 'Ducula bicolor',
# 'Ducula luctuosa',
# 'Ducula spilorrhoa',
# 'Cryptophaps poecilorrhoa',
# 'Gymnophaps albertisii',
# 'Gymnophaps mada',
# 'Gymnophaps stalkeri',
# 'Heliopais personatus',
# 'Rallicula leucospila',
# 'Rallicula rubra',
# 'Rallicula forbesi',
# 'Rallicula mayri',
# 'Rallus aquaticus',
# 'Rallus indicus',
# 'Crex crex',
# 'Aramidopsis plateni',
# 'Lewinia striata',
# 'Lewinia mirifica',
# 'Lewinia pectoralis',
# 'Habroptila wallacii',
# 'Gallirallus calayanensis',
# 'Eulabeornis castaneoventris',
# 'Hypotaenidia okinawae',
# 'Hypotaenidia torquata',
# 'Hypotaenidia philippensis',
# 'Porzana carolina',
# 'Porzana porzana',
# 'Gallinula tenebrosa',
# 'Gallinula chloropus',
# 'Fulica atra',
# 'Porphyrio alleni',
# 'Porphyrio porphyrio',
# 'Coturnicops exquisitus',
# 'Zapornia fusca',
# 'Zapornia paykullii',
# 'Zapornia bicolor',
# 'Zapornia akool',
# 'Zapornia pusilla',
# 'Zapornia parva',
# 'Zapornia tabuensis',
# 'Rallina eurizonoides',
# 'Rallina canningi',
# 'Rallina fasciata',
# 'Rallina tricolor',
# 'Gymnocrex rosenbergii',
# 'Gymnocrex talaudensis',
# 'Gymnocrex plumbeiventris',
# 'Megacrex inepta',
# 'Poliolimnas cinereus',
# 'Gallicrex cinerea',
# 'Amaurornis phoenicurus',
# 'Amaurornis olivacea',
# 'Amaurornis magnirostris',
# 'Amaurornis isabellina',
# 'Amaurornis moluccana',
# 'Leucogeranus leucogeranus',
# 'Antigone canadensis',
# 'Antigone vipio',
# 'Antigone antigone',
# 'Antigone rubicunda',
# 'Grus virgo',
# 'Grus japonensis',
# 'Grus grus',
# 'Grus monacha',
# 'Grus nigricollis',
# 'Tachybaptus ruficollis',
# 'Tachybaptus tricolor',
# 'Tachybaptus novaehollandiae',
# 'Podiceps grisegena',
# 'Podiceps cristatus',
# 'Podiceps auritus',
# 'Podiceps nigricollis',
# 'Phoenicopterus roseus',
# 'Phoeniconaias minor',
# 'Turnix sylvaticus',
# 'Turnix maculosus',
# 'Turnix tanki',
# 'Turnix ocellatus',
# 'Turnix suscitator',
# 'Turnix worcesteri',
# 'Turnix everetti',
# 'Burhinus oedicnemus',
# 'Burhinus indicus',
# 'Burhinus capensis',
# 'Burhinus grallarius',
# 'Esacus recurvirostris',
# 'Esacus magnirostris',
# 'Haematopus bachmani',
# 'Haematopus ostralegus',
# 'Haematopus longirostris',
# 'Haematopus fuliginosus',
# 'Ibidorhyncha struthersii',
# 'Himantopus himantopus',
# 'Himantopus leucocephalus',
# 'Recurvirostra avosetta',
# 'Vanellus vanellus',
# 'Vanellus spinosus',
# 'Vanellus duvaucelii',
# 'Vanellus tectus',
# 'Vanellus malabaricus',
# 'Vanellus cinereus',
# 'Vanellus indicus',
# 'Vanellus miles',
# 'Vanellus gregarius',
# 'Vanellus leucurus',
# 'Erythrogonys cinctus',
# 'Pluvialis apricaria',
# 'Pluvialis fulva',
# 'Pluvialis dominica',
# 'Pluvialis squatarola',
# 'Charadrius hiaticula',
# 'Charadrius semipalmatus',
# 'Charadrius placidus',
# 'Charadrius dubius',
# 'Charadrius vociferus',
# 'Charadrius pecuarius',
# 'Charadrius alexandrinus',
# 'Charadrius nivosus',
# 'Charadrius javanicus',
# 'Charadrius ruficapillus',
# 'Charadrius peronii',
# 'Charadrius mongolus',
# 'Charadrius leschenaultii',
# 'Charadrius asiaticus',
# 'Charadrius veredus',
# 'Charadrius morinellus',
# 'Elseyornis melanops',
# 'Pluvianus aegyptius',
# 'Rostratula benghalensis',
# 'Irediparra gallinacea',
# 'Hydrophasianus chirurgus',
# 'Metopidius indicus',
# 'Bartramia longicauda',
# 'Numenius tahitiensis',
# 'Numenius phaeopus',
# 'Numenius minutus',
# 'Numenius borealis',
# 'Numenius madagascariensis',
# 'Numenius tenuirostris',
# 'Numenius arquata',
# 'Limosa lapponica',
# 'Limosa limosa',
# 'Limosa haemastica',
# 'Arenaria interpres',
# 'Arenaria melanocephala',
# 'Calidris tenuirostris',
# 'Calidris canutus',
# 'Calidris pugnax',
# 'Calidris falcinellus',
# 'Calidris acuminata',
# 'Calidris himantopus',
# 'Calidris ferruginea',
# 'Calidris temminckii',
# 'Calidris subminuta',
# 'Calidris pygmaea',
# 'Calidris ruficollis',
# 'Calidris alba',
# 'Calidris alpina',
# 'Calidris ptilocnemis',
# 'Calidris maritima',
# 'Calidris bairdii',
# 'Calidris minuta',
# 'Calidris minutilla',
# 'Calidris fuscicollis',
# 'Calidris subruficollis',
# 'Calidris melanotos',
# 'Calidris pusilla',
# 'Calidris mauri',
# 'Limnodromus semipalmatus',
# 'Limnodromus scolopaceus',
# 'Limnodromus griseus',
# 'Scolopax rusticola',
# 'Scolopax mira',
# 'Scolopax saturata',
# 'Scolopax rosenbergii',
# 'Scolopax bukidnonensis',
# 'Scolopax celebensis',
# 'Scolopax rochussenii',
# 'Lymnocryptes minimus',
# 'Gallinago solitaria',
# 'Gallinago hardwickii',
# 'Gallinago nemoricola',
# 'Gallinago stenura',
# 'Gallinago megala',
# 'Gallinago media',
# 'Gallinago gallinago',
# 'Xenus cinereus',
# 'Phalaropus tricolor',
# 'Phalaropus lobatus',
# 'Phalaropus fulicarius',
# 'Actitis hypoleucos',
# 'Actitis macularius',
# 'Tringa ochropus',
# 'Tringa solitaria',
# 'Tringa incana',
# 'Tringa brevipes',
# 'Tringa flavipes',
# 'Tringa totanus',
# 'Tringa stagnatilis',
# 'Tringa glareola',
# 'Tringa erythropus',
# 'Tringa nebularia',
# 'Tringa guttifer',
# 'Tringa melanoleuca',
# 'Dromas ardeola',
# 'Cursorius cursor',
# 'Cursorius coromandelicus',
# 'Rhinoptilus bitorquatus',
# 'Stiltia isabella',
# 'Glareola pratincola',
# 'Glareola maldivarum',
# 'Glareola nordmanni',
# 'Glareola lactea',
# 'Anous stolidus',
# 'Anous tenuirostris',
# 'Anous minutus',
# 'Anous ceruleus',
# 'Gygis alba',
# 'Rynchops flavirostris',
# 'Rynchops albicollis',
# 'Rissa tridactyla',
# 'Rissa brevirostris',
# 'Pagophila eburnea',
# 'Xema sabini',
# 'Chroicocephalus genei',
# 'Chroicocephalus philadelphia',
# 'Chroicocephalus novaehollandiae',
# 'Chroicocephalus brunnicephalus',
# 'Chroicocephalus ridibundus',
# 'Chroicocephalus cirrocephalus',
# 'Chroicocephalus saundersi',
# 'Hydrocoloeus minutus',
# 'Rhodostethia rosea',
# 'Leucophaeus atricilla',
# 'Leucophaeus pipixcan',
# 'Ichthyaetus relictus',
# 'Ichthyaetus audouinii',
# 'Ichthyaetus melanocephalus',
# 'Ichthyaetus ichthyaetus',
# 'Ichthyaetus leucophthalmus',
# 'Ichthyaetus hemprichii',
# 'Larus crassirostris',
# 'Larus canus',
# 'Larus delawarensis',
# 'Larus californicus',
# 'Larus marinus',
# 'Larus glaucescens',
# 'Larus hyperboreus',
# 'Larus glaucoides',
# 'Larus argentatus',
# 'Larus smithsonianus',
# 'Larus vegae',
# 'Larus cachinnans',
# 'Larus armenicus',
# 'Larus schistisagus',
# 'Larus fuscus',
# 'Gelochelidon nilotica',
# 'Hydroprogne caspia',
# 'Thalasseus bergii',
# 'Thalasseus bengalensis',
# 'Thalasseus bernsteini',
# 'Thalasseus sandvicensis',
# 'Sternula albifrons',
# 'Sternula saundersi',
# 'Sternula antillarum',
# 'Onychoprion aleuticus',
# 'Onychoprion lunatus',
# 'Onychoprion anaethetus',
# 'Onychoprion fuscatus',
# 'Sterna aurantia',
# 'Sterna dougallii',
# 'Sterna sumatrana',
# 'Sterna hirundo',
# 'Sterna repressa',
# 'Sterna paradisaea',
# 'Sterna acuticauda',
# 'Chlidonias hybrida',
# 'Chlidonias leucopterus',
# 'Chlidonias niger',
# 'Stercorarius maccormicki',
# 'Stercorarius antarcticus',
# 'Stercorarius skua',
# 'Stercorarius pomarinus',
# 'Stercorarius parasiticus',
# 'Stercorarius longicaudus',
# 'Alle alle',
# 'Uria lomvia',
# 'Uria aalge',
# 'Alca torda',
# 'Cepphus grylle',
# 'Cepphus columba',
# 'Cepphus carbo',
# 'Brachyramphus marmoratus',
# 'Brachyramphus perdix',
# 'Brachyramphus brevirostris',
# 'Synthliboramphus antiquus',
# 'Synthliboramphus wumizusume',
# 'Ptychoramphus aleuticus',
# 'Aethia psittacula',
# 'Aethia pusilla',
# 'Aethia pygmaea',
# 'Aethia cristatella',
# 'Cerorhinca monocerata',
# 'Fratercula arctica',
# 'Fratercula corniculata',
# 'Fratercula cirrhata',
# 'Phaethon aethereus',
# 'Phaethon rubricauda',
# 'Phaethon lepturus',
# 'Gavia stellata',
# 'Gavia arctica',
# 'Gavia pacifica',
# 'Gavia immer',
# 'Gavia adamsii',
# 'Oceanites oceanicus',
# 'Pelagodroma marina',
# 'Fregetta grallaria',
# 'Fregetta tropica',
# 'Phoebastria immutabilis',
# 'Phoebastria nigripes',
# 'Phoebastria albatrus',
# 'Diomedea exulans',
# 'Thalassarche cauta',
# 'Thalassarche carteri',
# 'Hydrobates pelagicus',
# 'Hydrobates furcatus',
# 'Hydrobates monorhis',
# 'Hydrobates matsudairae',
# 'Hydrobates leucorhous',
# 'Hydrobates castro',
# 'Hydrobates tristrami',
# 'Hydrobates microsoma',
# 'Macronectes giganteus',
# 'Fulmarus glacialis',
# 'Daption capense',
# 'Pachyptila desolata',
# 'Pachyptila belcheri',
# 'Pterodroma incerta',
# 'Pterodroma solandri',
# 'Pterodroma externa',
# 'Pterodroma neglecta',
# 'Pterodroma baraui',
# 'Pterodroma sandwichensis',
# 'Pterodroma inexpectata',
# 'Pterodroma cervicalis',
# 'Pterodroma nigripennis',
# 'Pterodroma hypoleuca',
# 'Pterodroma leucoptera',
# 'Pterodroma longirostris',
# 'Pseudobulweria aterrima',
# 'Pseudobulweria rostrata',
# 'Calonectris leucomelas',
# 'Calonectris borealis',
# 'Ardenna pacifica',
# 'Ardenna bulleri',
# 'Ardenna grisea',
# 'Ardenna tenuirostris',
# 'Ardenna carneipes',
# 'Ardenna gravis',
# 'Puffinus nativitatis',
# 'Puffinus puffinus',
# 'Puffinus yelkouan',
# 'Puffinus lherminieri',
# 'Puffinus persicus',
# 'Puffinus bailloni',
# 'Puffinus bannermani',
# 'Bulweria bulwerii',
# 'Bulweria fallax',
# 'Mycteria cinerea',
# 'Mycteria ibis',
# 'Mycteria leucocephala',
# 'Anastomus oscitans',
# 'Ciconia nigra',
# 'Ciconia abdimii',
# 'Ciconia episcopus',
# 'Ciconia stormi',
# 'Ciconia ciconia',
# 'Ciconia boyciana',
# 'Ephippiorhynchus asiaticus',
# 'Leptoptilos javanicus',
# 'Leptoptilos dubius',
# 'Leptoptilos crumenifer',
# 'Fregata andrewsi',
# 'Fregata minor',
# 'Fregata ariel',
# 'Morus bassanus',
# 'Papasula abbotti',
# 'Sula dactylatra',
# 'Sula sula',
# 'Sula leucogaster',
# 'Anhinga melanogaster',
# 'Anhinga novaehollandiae',
# 'Microcarbo pygmaeus',
# 'Microcarbo niger',
# 'Microcarbo melanoleucos',
# 'Urile urile',
# 'Urile pelagicus',
# 'Phalacrocorax nigrogularis',
# 'Phalacrocorax sulcirostris',
# 'Phalacrocorax fuscicollis',
# 'Phalacrocorax capillatus',
# 'Phalacrocorax carbo',
# 'Gulosus aristotelis',
# 'Nannopterum auritum',
# 'Threskiornis aethiopicus',
# 'Threskiornis melanocephalus',
# 'Threskiornis molucca',
# 'Threskiornis spinicollis',
# 'Pseudibis papillosa',
# 'Pseudibis davisoni',
# 'Pseudibis gigantea',
# 'Geronticus eremita',
# 'Nipponia nippon',
# 'Plegadis falcinellus',
# 'Platalea leucorodia',
# 'Platalea minor',
# 'Platalea alba',
# 'Platalea regia',
# 'Zonerodius heliosylus',
# 'Botaurus stellaris',
# 'Ixobrychus minutus',
# 'Ixobrychus dubius',
# 'Ixobrychus sinensis',
# 'Ixobrychus eurhythmus',
# 'Ixobrychus cinnamomeus',
# 'Ixobrychus flavicollis',
# 'Gorsachius magnificus',
# 'Gorsachius goisagi',
# 'Gorsachius melanolophus',
# 'Nycticorax nycticorax',
# 'Nycticorax caledonicus',
# 'Butorides striata',
# 'Ardeola ralloides',
# 'Ardeola grayii',
# 'Ardeola bacchus',
# 'Ardeola speciosa',
# 'Bubulcus ibis',
# 'Bubulcus coromandus',
# 'Ardea cinerea',
# 'Ardea pacifica',
# 'Ardea melanocephala',
# 'Ardea insignis',
# 'Ardea sumatrana',
# 'Ardea goliath',
# 'Ardea purpurea',
# 'Ardea alba',
# 'Egretta picata',
# 'Egretta novaehollandiae',
# 'Egretta garzetta',
# 'Egretta gularis',
# 'Egretta sacra',
# 'Egretta eulophotes',
# 'Scopus umbretta',
# 'Pelecanus onocrotalus',
# 'Pelecanus rufescens',
# 'Pelecanus philippensis',
# 'Pelecanus crispus',
# 'Pelecanus conspicillatus',
# 'Pandion haliaetus',
# 'Pandion cristatus',
# 'Elanus caeruleus',
# 'Chelictinia riocourii',
# 'Gypaetus barbatus',
# 'Neophron percnopterus',
# 'Pernis apivorus',
# 'Pernis ptilorhynchus',
# 'Pernis celebensis',
# 'Pernis steerei',
# 'Aviceda jerdoni',
# 'Aviceda subcristata',
# 'Aviceda leuphotes',
# 'Henicopernis longicauda',
# 'Gyps bengalensis',
# 'Gyps indicus',
# 'Gyps tenuirostris',
# 'Gyps rueppelli',
# 'Gyps himalayensis',
# 'Gyps fulvus',
# 'Sarcogyps calvus',
# 'Aegypius monachus',
# 'Torgos tracheliotos',
# 'Spilornis cheela',
# 'Spilornis klossi',
# 'Spilornis kinabaluensis',
# 'Spilornis rufipectus',
# 'Spilornis holospilus',
# 'Spilornis elgini',
# 'Pithecophaga jefferyi',
# 'Circaetus gallicus',
# 'Terathopius ecaudatus',
# 'Macheiramphus alcinus',
# 'Harpyopsis novaeguineae',
# 'Nisaetus cirrhatus',
# 'Nisaetus floris',
# 'Nisaetus nipalensis',
# 'Nisaetus kelaarti',
# 'Nisaetus alboniger',
# 'Nisaetus bartelsi',
# 'Nisaetus lanceolatus',
# 'Nisaetus philippensis',
# 'Nisaetus pinskeri',
# 'Nisaetus nanus',
# 'Lophotriorchis kienerii',
# 'Ictinaetus malaiensis',
# 'Clanga pomarina',
# 'Clanga hastata',
# 'Clanga clanga',
# 'Hieraaetus pennatus',
# 'Hieraaetus weiskei',
# 'Aquila rapax',
# 'Aquila nipalensis',
# 'Aquila heliaca',
# 'Aquila gurneyi',
# 'Aquila chrysaetos',
# 'Aquila audax',
# 'Aquila verreauxii',
# 'Aquila fasciata',
# 'Micronisus gabar',
# 'Melierax metabates',
# 'Erythrotriorchis buergersi',
# 'Megatriorchis doriae',
# 'Accipiter trivirgatus',
# 'Accipiter griseiceps',
# 'Accipiter badius',
# 'Accipiter butleri',
# 'Accipiter brevipes',
# 'Accipiter soloensis',
# 'Accipiter trinotatus',
# 'Accipiter novaehollandiae',
# 'Accipiter hiogaster',
# 'Accipiter fasciatus',
# 'Accipiter melanochlamys',
# 'Accipiter henicogrammus',
# 'Accipiter poliocephalus',
# 'Accipiter gularis',
# 'Accipiter virgatus',
# 'Accipiter nanus',
# 'Accipiter erythrauchen',
# 'Accipiter cirrocephalus',
# 'Accipiter rhodogaster',
# 'Accipiter nisus',
# 'Accipiter gentilis',
# 'Accipiter meyerianus',
# 'Circus aeruginosus',
# 'Circus spilonotus',
# 'Circus approximans',
# 'Circus assimilis',
# 'Circus cyaneus',
# 'Circus macrourus',
# 'Circus melanoleucos',
# 'Circus pygargus',
# 'Milvus milvus',
# 'Milvus migrans',
# 'Haliastur sphenurus',
# 'Haliastur indus',
# 'Haliaeetus leucogaster',
# 'Haliaeetus leucoryphus',
# 'Haliaeetus albicilla',
# 'Haliaeetus leucocephalus',
# 'Haliaeetus pelagicus',
# 'Haliaeetus humilis',
# 'Haliaeetus ichthyaetus',
# 'Butastur teesa',
# 'Butastur liventer',
# 'Butastur indicus',
# 'Buteo lagopus',
# 'Buteo hemilasius',
# 'Buteo japonicus',
# 'Buteo refectus',
# 'Buteo rufinus',
# 'Buteo buteo',
# 'Tyto tenebricosa',
# 'Tyto inexspectata',
# 'Tyto nigrobrunnea',
# 'Tyto sororcula',
# 'Tyto novaehollandiae',
# 'Tyto rosenbergii',
# 'Tyto alba',
# 'Tyto deroepstorffi',
# 'Tyto longimembris',
# 'Phodilus badius',
# 'Phodilus assimilis',
# 'Uroglaux dimorpha',
# 'Ninox rufa',
# 'Ninox connivens',
# 'Ninox rudolfi',
# 'Ninox boobook',
# 'Ninox novaeseelandiae',
# 'Ninox japonica',
# 'Ninox scutulata',
# 'Ninox obscura',
# 'Ninox randi',
# 'Ninox affinis',
# 'Ninox philippensis',
# 'Ninox sumbaensis',
# 'Ninox burhani',
# 'Ninox ochracea',
# 'Ninox ios',
# 'Ninox hypogramma',
# 'Ninox squamipila',
# 'Ninox forbesi',
# 'Ninox natalis',
# 'Ninox theomacha',
# 'Ninox punctulata',
# 'Taenioptynx brodiei',
# 'Aegolius funereus',
# 'Athene brama',
# 'Athene noctua',
# 'Athene blewitti',
# 'Surnia ulula',
# 'Glaucidium passerinum',
# 'Glaucidium cuculoides',
# 'Glaucidium castanopterum',
# 'Glaucidium radiatum',
# 'Glaucidium castanotum',
# 'Otus gurneyi',
# 'Otus sagittatus',
# 'Otus rufescens',
# 'Otus thilohoffmanni',
# 'Otus balli',
# 'Otus alfredi',
# 'Otus spilocephalus',
# 'Otus angelinae',
# 'Otus mirus',
# 'Otus longicornis',
# 'Otus mindorensis',
# 'Otus scops',
# 'Otus brucei',
# 'Otus sunia',
# 'Otus elegans',
# 'Otus magicus',
# 'Otus sulaensis',
# 'Otus beccarii',
# 'Otus manadensis',
# 'Otus siaoensis',
# 'Otus collari',
# 'Otus mantananensis',
# 'Otus alius',
# 'Otus umbra',
# 'Otus enganensis',
# 'Otus mentawi',
# 'Otus brookii',
# 'Otus bakkamoena',
# 'Otus lettia',
# 'Otus semitorques',
# 'Otus lempiji',
# 'Otus megalotis',
# 'Otus nigrorum',
# 'Otus everetti',
# 'Otus fuliginosus',
# 'Otus silvicola',
# 'Asio otus',
# 'Asio flammeus',
# 'Bubo scandiacus',
# 'Bubo bubo',
# 'Bubo bengalensis',
# 'Bubo ascalaphus',
# 'Bubo sumatranus',
# 'Bubo nipalensis',
# 'Bubo coromandus',
# 'Bubo philippensis',
# 'Bubo blakistoni',
# 'Ketupa zeylonensis',
# 'Ketupa flavipes',
# 'Ketupa ketupu',
# 'Strix seloputo',
# 'Strix ocellata',
# 'Strix leptogrammica',
# 'Strix aluco',
# 'Strix nivicolum',
# 'Strix butleri',
# 'Strix uralensis',
# 'Strix davidi',
# 'Strix nebulosa',
# 'Apalharpactes reinwardtii',
# 'Apalharpactes mackloti',
# 'Harpactes fasciatus',
# 'Harpactes kasumba',
# 'Harpactes diardii',
# 'Harpactes ardens',
# 'Harpactes whiteheadi',
# 'Harpactes orrhophaeus',
# 'Harpactes duvaucelii',
# 'Harpactes oreskios',
# 'Harpactes erythrocephalus',
# 'Harpactes wardi',
# 'Upupa epops',
# 'Lophoceros nasutus',
# 'Berenicornis comatus',
# 'Buceros rhinoceros',
# 'Buceros bicornis',
# 'Rhinoplax vigil',
# 'Anthracoceros marchei',
# 'Anthracoceros albirostris',
# 'Anthracoceros coronatus',
# 'Anthracoceros montani',
# 'Anthracoceros malayanus',
# 'Ocyceros griseus',
# 'Ocyceros gingalensis',
# 'Ocyceros birostris',
# 'Anorrhinus tickelli',
# 'Anorrhinus austeni',
# 'Anorrhinus galeritus',
# 'Aceros nipalensis',
# 'Rhyticeros plicatus',
# 'Rhyticeros narcondami',
# 'Rhyticeros undulatus',
# 'Rhyticeros everetti',
# 'Rhyticeros subruficollis',
# 'Rhyticeros cassidix',
# 'Rhabdotorrhinus waldeni',
# 'Rhabdotorrhinus leucocephalus',
# 'Rhabdotorrhinus exarhatus',
# 'Rhabdotorrhinus corrugatus',
# 'Penelopides manillae',
# 'Penelopides mindorensis',
# 'Penelopides affinis',
# 'Penelopides samarensis',
# 'Penelopides panini',
# 'Coracias naevius',
# 'Coracias benghalensis',
# 'Coracias temminckii',
# 'Coracias caudatus',
# 'Coracias abyssinicus',
# 'Coracias garrulus',
# 'Eurystomus glaucurus',
# 'Eurystomus orientalis',
# 'Eurystomus azureus',
# 'Actenoides monachus',
# 'Actenoides princeps',
# 'Actenoides lindsayi',
# 'Actenoides hombroni',
# 'Actenoides concretus',
# 'Melidora macrorrhina',
# 'Lacedo pulchella',
# 'Tanysiptera galatea',
# 'Tanysiptera ellioti',
# 'Tanysiptera riedelii',
# 'Tanysiptera carolinae',
# 'Tanysiptera hydrocharis',
# 'Tanysiptera nympha',
# 'Tanysiptera danae',
# 'Cittura cyanotis',
# 'Clytoceyx rex',
# 'Dacelo leachii',
# 'Dacelo tyro',
# 'Dacelo gaudichaud',
# 'Caridonax fulgidus',
# 'Pelargopsis capensis',
# 'Pelargopsis melanorhyncha',
# 'Pelargopsis amauroptera',
# 'Halcyon coromanda',
# 'Halcyon smyrnensis',
# 'Halcyon cyanoventris',
# 'Halcyon pileata',
# 'Halcyon leucocephala',
# 'Todiramphus nigrocyaneus',
# 'Todiramphus winchelli',
# 'Todiramphus diops',
# 'Todiramphus lazuli',
# 'Todiramphus macleayii',
# 'Todiramphus funebris',
# 'Todiramphus chloris',
# 'Todiramphus enigma',
# 'Todiramphus saurophagus',
# 'Todiramphus sanctus',
# 'Todiramphus australasia',
# 'Syma torotoro',
# 'Syma megarhyncha',
# 'Corythornis cristatus',
# 'Alcedo coerulescens',
# 'Alcedo euryzona',
# 'Alcedo meninting',
# 'Alcedo atthis',
# 'Alcedo hercules',
# 'Ceyx erithaca',
# 'Ceyx melanurus',
# 'Ceyx fallax',
# 'Ceyx lepidus',
# 'Ceyx cyanopectus',
# 'Ceyx argentatus',
# 'Ceyx azureus',
# 'Ceyx pusillus',
# 'Megaceryle lugubris',
# 'Ceryle rudis',
# 'Nyctyornis amictus',
# 'Nyctyornis athertoni',
# 'Meropogon forsteni',
# 'Merops albicollis',
# 'Merops orientalis',
# 'Merops persicus',
# 'Merops philippinus',
# 'Merops ornatus',
# 'Merops viridis',
# 'Merops leschenaulti',
# 'Merops apiaster',
# 'Psilopogon pyrolophus',
# 'Psilopogon virens',
# 'Psilopogon lagrandieri',
# 'Psilopogon zeylanicus',
# 'Psilopogon lineatus',
# 'Psilopogon viridis',
# 'Psilopogon faiostrictus',
# 'Psilopogon corvinus',
# 'Psilopogon chrysopogon',
# 'Psilopogon rafflesii',
# 'Psilopogon mystacophanos',
# 'Psilopogon javensis',
# 'Psilopogon flavifrons',
# 'Psilopogon franklinii',
# 'Psilopogon oorti',
# 'Psilopogon annamensis',
# 'Psilopogon faber',
# 'Psilopogon nuchalis',
# 'Psilopogon asiaticus',
# 'Psilopogon monticola',
# 'Psilopogon incognitus',
# 'Psilopogon henricii',
# 'Psilopogon armillaris',
# 'Psilopogon pulcherrimus',
# 'Psilopogon australis',
# 'Psilopogon eximius',
# 'Psilopogon rubricapillus',
# 'Psilopogon malabaricus',
# 'Psilopogon haemacephalus',
# 'Caloramphus fuliginosus',
# 'Indicator xanthonotus',
# 'Indicator archipelagicus',
# 'Jynx torquilla',
# 'Picumnus innominatus',
# 'Sasia abnormis',
# 'Sasia ochracea',
# 'Hemicircus concretus',
# 'Hemicircus canente',
# 'Yungipicus temminckii',
# 'Yungipicus canicapillus',
# 'Yungipicus ramsayi',
# 'Yungipicus kizuki',
# 'Picoides tridactylus',
# 'Dendrocoptes auriceps',
# 'Dendrocoptes medius',
# 'Dryobates minor',
# 'Dendrocopos hyperythrus',
# 'Dendrocopos macei',
# 'Dendrocopos analis',
# 'Dendrocopos atratus',
# 'Dendrocopos darjellensis',
# 'Dendrocopos himalayensis',
# 'Dendrocopos assimilis',
# 'Dendrocopos syriacus',
# 'Dendrocopos leucopterus',
# 'Dendrocopos major',
# 'Dendrocopos noguchii',
# 'Dendrocopos leucotos',
# 'Dryocopus javensis',
# 'Dryocopus hodgei',
# 'Dryocopus martius',
# 'Chrysophlegma miniaceum',
# 'Chrysophlegma mentale',
# 'Chrysophlegma flavinucha',
# 'Picus chlorolophus',
# 'Picus puniceus',
# 'Picus viridanus',
# 'Picus vittatus',
# 'Picus xanthopygaeus',
# 'Picus squamatus',
# 'Picus awokera',
# 'Picus viridis',
# 'Picus rabieri',
# 'Picus erythropygius',
# 'Picus canus',
# 'Dinopium shorii',
# 'Dinopium javanense',
# 'Dinopium benghalense',
# 'Chrysocolaptes lucidus',
# 'Chrysocolaptes haematribon',
# 'Chrysocolaptes xanthocephalus',
# 'Chrysocolaptes erythrocephalus',
# 'Chrysocolaptes strictus',
# 'Chrysocolaptes guttacristatus',
# 'Chrysocolaptes stricklandi',
# 'Chrysocolaptes festivus',
# 'Gecinulus grantia',
# 'Gecinulus viridis',
# 'Gecinulus rafflesii',
# 'Blythipicus rubiginosus',
# 'Blythipicus pyrrhotis',
# 'Reinwardtipicus validus',
# 'Micropternus brachyurus',
# 'Meiglyptes tristis',
# 'Meiglyptes jugularis',
# 'Meiglyptes tukki',
# 'Mulleripicus fulvus',
# 'Mulleripicus funebris',
# 'Mulleripicus pulverulentus',
# 'Polihierax insignis',
# 'Microhierax caerulescens',
# 'Microhierax fringillarius',
# 'Microhierax latifrons',
# 'Microhierax erythrogenys',
# 'Microhierax melanoleucos',
# 'Falco naumanni',
# 'Falco tinnunculus',
# 'Falco moluccensis',
# 'Falco cenchroides',
# 'Falco sparverius',
# 'Falco chicquera',
# 'Falco vespertinus',
# 'Falco amurensis',
# 'Falco eleonorae',
# 'Falco concolor',
# 'Falco columbarius',
# 'Falco subbuteo',
# 'Falco severus',
# 'Falco longipennis',
# 'Falco berigora',
# 'Falco hypoleucos',
# 'Falco biarmicus',
# 'Falco jugger',
# 'Falco cherrug',
# 'Falco rusticolus',
# 'Falco peregrinus',
# 'Probosciger aterrimus',
# 'Cacatua pastinator',
# 'Cacatua sanguinea',
# 'Cacatua goffiniana',
# 'Cacatua haematuropygia',
# 'Cacatua galerita',
# 'Cacatua sulphurea',
# 'Cacatua moluccensis',
# 'Cacatua alba',
# 'Psittrichas fulgidus',
# 'Micropsitta keiensis',
# 'Micropsitta geelvinkiana',
# 'Micropsitta pusio',
# 'Micropsitta bruijnii',
# 'Alisterus amboinensis',
# 'Alisterus chloropterus',
# 'Aprosmictus jonquillaceus',
# 'Aprosmictus erythropterus',
# 'Prioniturus mada',
# 'Prioniturus platurus',
# 'Prioniturus waterstradti',
# 'Prioniturus montanus',
# 'Prioniturus platenae',
# 'Prioniturus verticalis',
# 'Prioniturus flavicans',
# 'Prioniturus luconensis',
# 'Prioniturus discurus',
# 'Eclectus roratus',
# 'Geoffroyus geoffroyi',
# 'Geoffroyus simplex',
# 'Psittinus cyanurus',
# 'Tanygnathus megalorynchos',
# 'Tanygnathus lucionensis',
# 'Tanygnathus sumatranus',
# 'Tanygnathus gramineus',
# 'Psittacula finschii',
# 'Psittacula himalayana',
# 'Psittacula roseata',
# 'Psittacula cyanocephala',
# 'Psittacula alexandri',
# 'Psittacula derbiana',
# 'Psittacula longicauda',
# 'Psittacula columboides',
# 'Psittacula calthrapae',
# 'Psittacula eupatria',
# 'Psittacula krameri',
# 'Psittacula caniceps',
# 'Psittacella brehmii',
# 'Psittacella picta',
# 'Psittacella modesta',
# 'Psittacella madaraszi',
# 'Oreopsittacus arfaki',
# 'Charminetta wilhelminae',
# 'Hypocharmosyna rubronotata',
# 'Hypocharmosyna placentis',
# 'Charmosynopsis toxopei',
# 'Charmosynopsis pulchella',
# 'Synorhacma multistriata',
# 'Charmosyna josefinae',
# 'Charmosyna papou',
# 'Vini rubrigularis',
# 'Neopsittacus musschenbroekii',
# 'Neopsittacus pullicauda',
# 'Lorius domicella',
# 'Lorius garrulus',
# 'Lorius lory',
# 'Pseudeos fuscata',
# 'Chalcopsitta duivenbodei',
# 'Chalcopsitta atra',
# 'Chalcopsitta scintillata',
# 'Glossoptilus goldiei',
# 'Saudareos johnstoniae',
# 'Saudareos iris',
# 'Saudareos flavoviridis',
# 'Saudareos ornatus',
# 'Eos reticulata',
# 'Eos semilarvata',
# 'Eos bornea',
# 'Eos cyanogenia',
# 'Eos histrio',
# 'Eos squamata',
# 'Trichoglossus haematodus',
# 'Trichoglossus rosenbergii',
# 'Trichoglossus rubritorquis',
# 'Trichoglossus euteles',
# 'Trichoglossus capistratus',
# 'Trichoglossus weberi',
# 'Trichoglossus forsteni',
# 'Melopsittacus undulatus',
# 'Psittaculirostris desmarestii',
# 'Psittaculirostris edwardsii',
# 'Psittaculirostris salvadorii',
# 'Cyclopsitta gulielmitertii',
# 'Cyclopsitta diophthalma',
# 'Bolbopsittacus lunulatus',
# 'Loriculus vernalis',
# 'Loriculus beryllinus',
# 'Loriculus philippensis',
'Loriculus camiguinensis',
'Loriculus galgulus',
'Loriculus stigmatus',
'Loriculus amabilis',
'Loriculus sclateri',
'Loriculus catamene',
'Loriculus aurantiifrons',
'Loriculus exilis',
'Loriculus pusillus',
'Loriculus flosculus',
'Psarisomus dalhousiae',
'Corydon sumatranus',
'Sarcophanops samarensis',
'Sarcophanops steerii',
'Serilophus lunatus',
'Cymbirhynchus macrorhynchos',
'Eurylaimus javanicus',
'Eurylaimus ochromalus',
'Calyptomena viridis',
'Calyptomena hosii',
'Calyptomena whiteheadi',
'Hydrornis phayrei',
'Hydrornis caeruleus',
'Hydrornis oatesi',
'Hydrornis schneideri',
'Hydrornis nipalensis',
'Hydrornis soror',
'Hydrornis irena',
'Hydrornis guajanus',
'Hydrornis schwaneri',
'Hydrornis baudii',
'Hydrornis cyaneus',
'Hydrornis gurneyi',
'Hydrornis elliotii',
'Erythropitta kochi',
'Erythropitta erythrogaster',
'Erythropitta venusta',
'Erythropitta granatina',
'Erythropitta ussheri',
'Erythropitta arquata',
'Pitta brachyura',
'Pitta moluccensis',
'Pitta megarhyncha',
'Pitta sordida',
'Pitta nympha',
'Pitta versicolor',
'Pitta maxima',
'Pitta elegans',
'Pitta steerii',
'Ailuroedus buccoides',
'Ailuroedus melanotis',
'Archboldia papuensis',
'Amblyornis inornata',
'Amblyornis macgregoriae',
'Amblyornis subalaris',
'Amblyornis flavifrons',
'Sericulus aureus',
'Sericulus ardens',
'Sericulus bakeri',
'Chlamydera lauterbachi',
'Chlamydera cerviniventris',
'Cormobates placens',
'Sipodotus wallacii',
'Chenorhamphus grayi',
'Malurus cyanocephalus',
'Malurus alboscapulatus',
'Clytomyias insignis',
'Myza celebensis',
'Myza sarasinorum',
'Glycichaera fallax',
'Melionyx fuscus',
'Melionyx princeps',
'Melionyx nouhuysi',
'Ptiloprora plumbea',
'Ptiloprora meekiana',
'Ptiloprora erythropleura',
'Ptiloprora guisei',
'Ptiloprora mayri',
'Ptiloprora perstriata',
'Conopophila albogularis',
'Ramsayornis modestus',
'Melilestes megarhynchus',
'Timeliopsis fulvigula',
'Timeliopsis griseigula',
'Melipotes gymnops',
'Melipotes fumigatus',
'Melipotes carolae',
'Melipotes ater',
'Macgregoria pulchra',
'Pycnopygius ixoides',
'Pycnopygius cinereus',
'Pycnopygius stictocephalus',
'Melitograis gilolensis',
'Myzomela blasii',
'Myzomela albigula',
'Myzomela eques',
'Myzomela obscura',
'Myzomela cruentata',
'Myzomela nigrita',
'Myzomela kuehni',
'Myzomela erythrocephala',
'Myzomela dammermani',
'Myzomela adolphinae',
'Myzomela boiei',
'Myzomela chloroptera',
'Myzomela wakoloensis',
'Myzomela sanguinolenta',
'Myzomela sclateri',
'Myzomela vulnerata',
'Myzomela rosenbergii',
'Philemon meyeri',
'Philemon brassi',
'Philemon citreogularis',
'Philemon kisserensis',
'Philemon inornatus',
'Philemon fuscicapillus',
'Philemon subcorniculatus',
'Philemon moluccensis',
'Philemon plumigenis',
'Philemon buceroides',
'Philemon novaeguineae',
'Philemon corniculatus',
'Xanthotis polygrammus',
'Xanthotis flaviventer',
'Lichmera lombokia',
'Lichmera argentauris',
'Lichmera limbata',
'Lichmera indistincta',
'Lichmera alboauricularis',
'Lichmera squamata',
'Lichmera deningeri',
'Lichmera monticola',
'Lichmera flavicans',
'Lichmera notabilis',
'Entomyzon cyanotis',
'Melithreptus albogularis',
'Meliphaga aruensis',
'Microptilotis mimikae',
'Microptilotis montanus',
'Microptilotis orientalis',
'Microptilotis albonotatus',
'Microptilotis analogus',
'Microptilotis vicina',
'Microptilotis gracilis',
'Microptilotis flavirictus',
'Territornis reticulata',
'Gavicalis versicolor',
'Oreornis chrysogenys',
'Ptilotula flavescens',
'Caligavis subfrenata',
'Caligavis obscura',
'Melidectes ochromelas',
'Melidectes leucostephes',
'Melidectes rufocrissalis',
'Melidectes foersteri',
'Melidectes belfordi',
'Melidectes torquatus',
'Pachycare flavogriseum',
'Origma murina',
'Origma robusta',
'Aethomyias nigrorufus',
'Aethomyias spilodera',
'Aethomyias rufescens',
'Aethomyias perspicillatus',
'Aethomyias papuensis',
'Aethomyias arfakianus',
'Sericornis beccarii',
'Sericornis nouhuysi',
'Sericornis virgatus',
'Gerygone ruficollis',
'Gerygone sulphurea',
'Gerygone dorsalis',
'Gerygone levigaster',
'Gerygone inornata',
'Gerygone magnirostris',
'Gerygone hypoxantha',
'Gerygone chrysogaster',
'Gerygone chloronota',
'Gerygone olivacea',
'Gerygone palpebrosa',
'Acanthiza murina',
'Acanthiza cinerea',
'Garritornis isidorei',
'Pomatostomus temporalis',
'Orthonyx novaeguineae',
'Orthonyx temminckii',
'Cnemophilus loriae',
'Cnemophilus macgregorii',
'Loboparadisea sericea',
'Melanocharis arfakiana',
'Melanocharis longicauda',
'Melanocharis nigra',
'Melanocharis versteri',
'Melanocharis striativentris',
'Rhamphocharis crassirostris',
'Oedistoma iliolophus',
'Oedistoma pygmaeum',
'Toxorhamphus novaeguineae',
'Toxorhamphus poliopterus',
'Oreocharis arfaki',
'Paramythia montium',
'Androphobus viridis',
'Ptilorrhoa leucosticta',
'Ptilorrhoa caerulescens',
'Ptilorrhoa castanonota',
'Cinclosoma ajax',
'Tchagra senegalus',
'Machaerirhynchus flaviventer',
'Machaerirhynchus nigripectus',
'Hemipus picatus',
'Hemipus hirundinaceus',
'Tephrodornis virgatus',
'Tephrodornis sylvicola',
'Tephrodornis pondicerianus',
'Tephrodornis affinis',
'Philentoma pyrhoptera',
'Philentoma velata',
'Pityriasis gymnocephala',
'Artamus fuscus',
'Artamus leucorynchus',
'Artamus monachus',
'Artamus maximus',
'Artamus cinereus',
'Peltops blainvillii',
'Peltops montanus',
'Melloria quoyi',
'Gymnorhina tibicen',
'Cracticus mentalis',
'Cracticus cassicus',
'Cracticus louisiadensis',
'Rhagologus leucostigma',
'Aegithina tiphia',
'Aegithina nigrolutea',
'Aegithina viridissima',
'Aegithina lafresnayei',
'Pericrocotus erythropygius',
'Pericrocotus albifrons',
'Pericrocotus igneus',
'Pericrocotus cinnamomeus',
'Pericrocotus solaris',
'Pericrocotus miniatus',
'Pericrocotus brevirostris',
'Pericrocotus lansbergei',
'Pericrocotus ethologus',
'Pericrocotus flammeus',
'Pericrocotus speciosus',
'Pericrocotus divaricatus',
'Pericrocotus tegimae',
'Pericrocotus cantonensis',
'Pericrocotus roseus',
'Coracina caeruleogrisea',
'Coracina longicauda',
'Coracina temminckii',
'Coracina bicolor',
'Coracina lineata',
'Coracina novaehollandiae',
'Coracina boyeri',
'Coracina fortis',
'Coracina personata',
'Coracina striata',
'Coracina javensis',
'Coracina macei',
'Coracina dobsoni',
'Coracina schistacea',
'Coracina leucopygia',
'Coracina larvata',
'Coracina papuensis',
'Coracina atriceps',
'Campochaera sloetii',
'Malindangia mcgregori',
'Edolisoma ostentum',
'Edolisoma dohertyi',
'Edolisoma dispar',
'Edolisoma incertum',
'Edolisoma sula',
'Edolisoma tenuirostre',
'Edolisoma melas',
'Edolisoma parvulum',
'Celebesica abbotti',
'Lalage sueurii',
'Lalage aurea',
'Lalage atrovirens',
'Lalage moesta',
'Lalage leucomela',
'Lalage melanoleuca',
'Lalage nigra',
'Lalage leucopygialis',
'Lalage melaschistos',
'Lalage fimbriata',
'Daphoenositta chrysoptera',
'Daphoenositta papuensis',
'Daphoenositta miranda',
'Eulacestoma nigropectus',
'Aleadryas rufinucha',
'Ornorectes cristatus',
'Coracornis raveni',
'Coracornis sanghirensis',
'Melanorectes nigrescens',
'Pachycephala cinerea',
'Pachycephala albiventris',
'Pachycephala homeyeri',
'Pachycephala phaionota',
'Pachycephala hyperythra',
'Pachycephala modesta',
'Pachycephala philippinensis',
'Pachycephala sulfuriventer',
'Pachycephala hypoxantha',
'Pachycephala meyeri',
'Pachycephala simplex',
'Pachycephala orpheus',
'Pachycephala soror',
'Pachycephala fulvotincta',
'Pachycephala macrorhyncha',
'Pachycephala mentalis',
'Pachycephala pectoralis',
'Pachycephala melanura',
'Pachycephala nudigula',
'Pachycephala lorentzi',
'Pachycephala schlegelii',
'Pachycephala aurea',
'Pachycephala rufiventris',
'Pachycephala monacha',
'Pachycephala leucogastra',
'Pachycephala arctitorquis',
'Pachycephala griseonota',
'Pachycephala johni',
'Pachycephala tenebrosa',
'Pseudorectes incertus',
'Pseudorectes ferrugineus',
'Colluricincla tenebrosa',
'Colluricincla megarhyncha',
'Colluricincla harmonica',
'Lanius tigrinus',
'Lanius bucephalus',
'Lanius cristatus',
'Lanius collurio',
'Lanius isabellinus',
'Lanius phoenicuroides',
'Lanius collurioides',
'Lanius vittatus',
'Lanius schach',
'Lanius tephronotus',
'Lanius validirostris',
'Lanius minor',
'Lanius excubitor',
'Lanius meridionalis',
'Lanius sphenocercus',
'Lanius senator',
'Lanius nubicus',
'Pteruthius xanthochlorus',
'Pteruthius melanotis',
'Pteruthius rufiventer',
'Pteruthius ripleyi',
'Pteruthius aeralatus',
'Pteruthius annamensis',
'Pteruthius flaviscapis',
'Pteruthius aenobarbus',
'Erpornis zantholeuca',
'Sphecotheres viridis',
'Sphecotheres hypoleucus',
'Sphecotheres vieilloti',
'Pitohui kirhocephalus',
'Pitohui dichrous',
'Oriolus isabellae',
'Oriolus forsteni',
'Oriolus bouroensis',
'Oriolus phaeochromus',
'Oriolus sagittatus',
'Oriolus melanotis',
'Oriolus decipiens',
'Oriolus szalayi',
'Oriolus flavocinctus',
'Oriolus cruentus',
'Oriolus hosii',
'Oriolus traillii',
'Oriolus mellianus',
'Oriolus xanthornus',
'Oriolus xanthonotus',
'Oriolus steerii',
'Oriolus albiloris',
'Oriolus kundoo',
'Oriolus oriolus',
'Oriolus chinensis',
'Oriolus tenuirostris',
'Dicrurus aeneus',
'Dicrurus remifer',
'Dicrurus annectens',
'Dicrurus paradiseus',
'Dicrurus lophorinus',
'Dicrurus andamanensis',
'Dicrurus sumatranus',
'Dicrurus montanus',
'Dicrurus bracteatus',
'Dicrurus hottentottus',
'Dicrurus menagei',
'Dicrurus balicassius',
'Dicrurus densus',
'Dicrurus leucophaeus',
'Dicrurus caerulescens',
'Dicrurus macrocercus',
'Rhipidura superciliaris',
'Rhipidura cyaniceps',
'Rhipidura albicollis',
'Rhipidura albogularis',
'Rhipidura euryura',
'Rhipidura aureola',
'Rhipidura javanica',
'Rhipidura perlata',
'Rhipidura leucophrys',
'Rhipidura diluta',
'Rhipidura fuscorufa',
'Rhipidura rufiventris',
'Rhipidura threnothorax',
'Rhipidura maculipectus',
'Rhipidura leucothorax',
'Rhipidura atra',
'Rhipidura hyperythra',
'Rhipidura albolimbata',
'Rhipidura phasiana',
'Rhipidura phoenicura',
'Rhipidura nigrocinnamomea',
'Rhipidura brachyrhyncha',
'Rhipidura dedemi',
'Rhipidura superflua',
'Rhipidura teysmanni',
'Rhipidura opistherythra',
'Rhipidura rufidorsa',
'Rhipidura rufifrons',
'Rhipidura dryas',
'Chaetorhynchus papuensis',
'Eutrichomyias rowleyi',
'Hypothymis azurea',
'Hypothymis helenae',
'Hypothymis coelestis',
'Terpsiphone viridis',
'Terpsiphone paradisi',
'Terpsiphone atrocaudata',
'Terpsiphone cyanescens',
'Terpsiphone cinnamomea',
'Symposiachrus axillaris',
'Symposiachrus guttula',
'Symposiachrus mundus',
'Symposiachrus sacerdotum',
'Symposiachrus boanensis',
'Symposiachrus trivirgatus',
'Symposiachrus bimaculatus',
'Symposiachrus leucurus',
'Symposiachrus everetti',
'Symposiachrus loricatus',
'Symposiachrus julianae',
'Symposiachrus brehmii',
'Symposiachrus manadensis',
'Monarcha rubiensis',
'Monarcha cinerascens',
'Monarcha melanopsis',
'Monarcha frater',
'Carterornis pileatus',
'Carterornis chrysomela',
'Arses insularis',
'Arses telescopthalmus',
'Grallina cyanoleuca',
'Grallina bruijnii',
'Myiagra galeata',
'Myiagra atra',
'Myiagra rubecula',
'Myiagra ruficollis',
'Myiagra cyanoleuca',
'Myiagra alecto',
'Myiagra nana',
'Platylophus galericulatus',
'Platysmurus leucopterus',
'Perisoreus infaustus',
'Perisoreus internigrans',
'Garrulus glandarius',
'Garrulus lanceolatus',
'Garrulus lidthi',
'Cyanopica cyanus',
'Urocissa ornata',
'Urocissa caerulea',
'Urocissa flavirostris',
'Urocissa erythroryncha',
'Urocissa whiteheadi',
'Cissa chinensis',
'Cissa hypoleuca',
'Cissa thalassina',
'Cissa jefferyi',
'Dendrocitta vagabunda',
'Dendrocitta occipitalis',
'Dendrocitta cinerascens',
'Dendrocitta formosae',
'Dendrocitta leucogastra',
'Dendrocitta frontalis',
'Dendrocitta bayleii',
'Crypsirina temia',
'Crypsirina cucullata',
'Temnurus temnurus',
'Pica pica',
'Podoces hendersoni',
'Podoces biddulphi',
'Podoces panderi',
'Podoces pleskei',
'Nucifraga caryocatactes',
'Pyrrhocorax pyrrhocorax',
'Pyrrhocorax graculus',
'Coloeus monedula',
'Coloeus dauuricus',
'Corvus splendens',
'Corvus unicolor',
'Corvus enca',
'Corvus violaceus',
'Corvus typicus',
'Corvus florensis',
'Corvus validus',
'Corvus fuscicapillus',
'Corvus tristis',
'Corvus frugilegus',
'Corvus corone',
'Corvus cornix',
'Corvus torquatus',
'Corvus macrorhynchos',
'Corvus levaillantii',
'Corvus culminatus',
'Corvus orru',
'Corvus ruficollis',
'Corvus corax',
'Corvus rhipidurus',
'Melampitta lugubris',
'Megalampitta gigantea',
'Ifrita kowaldi',
'Lycocorax pyrrhopterus',
'Manucodia ater',
'Manucodia jobiensis',
'Manucodia chalybatus',
'Manucodia comrii',
'Phonygammus keraudrenii',
'Paradigalla carunculata',
'Paradigalla brevicauda',
'Astrapia nigra',
'Astrapia splendidissima',
'Astrapia mayeri',
'Astrapia stephaniae',
'Astrapia rothschildi',
'Parotia sefilata',
'Parotia carolae',
'Parotia berlepschi',
'Parotia lawesii',
'Parotia helenae',
'Parotia wahnesi',
'Pteridophora alberti',
'Lophorina superba',
'Ptiloris paradiseus',
'Ptiloris victoriae',
'Ptiloris magnificus',
'Ptiloris intercedens',
'Epimachus fastosus',
'Epimachus meyeri',
'Drepanornis albertisi',
'Drepanornis bruijnii',
'Diphyllodes magnificus',
'Diphyllodes respublica',
'Cicinnurus regius',
'Semioptera wallacii',
'Seleucidis melanoleucus',
'Paradisaea apoda',
'Paradisaea raggiana',
'Paradisaea minor',
'Paradisaea decora',
'Paradisaea rubra',
'Paradisaea guilielmi',
'Paradisaea rudolphi',
'Tregellasia leucops',
'Gennaeodryas placens',
'Peneothello sigillata',
'Peneothello cryptoleuca',
'Peneothello cyanus',
'Peneothello bimaculata',
'Peneothello pulverulenta',
'Poecilodryas brachyura',
'Poecilodryas hypoleuca',
'Plesiodryas albonotata',
'Heteromyias albispecularis',
'Heteromyias cinereifrons',
'Drymodes superciliaris',
'Drymodes brunneopygia',
'Microeca flavigaster',
'Microeca hemixantha',
'Microeca fascinans',
'Monachella muelleriana',
'Cryptomicroeca flaviventris',
'Kempiella griseoceps',
'Kempiella flavovirescens',
'Devioeca papuana',
'Eugerygone rubra',
'Petroica archboldi',
'Petroica bivittata',
'Pachycephalopsis poliosoma',
'Amalocichla sclateriana',
'Amalocichla incerta',
'Eupetes macrocerus',
'Bombycilla garrulus',
'Bombycilla japonica',
'Hypocolius ampelinus',
'Hylocitrea bonensis',
'Chelidorhynx hypoxanthus',
'Culicicapa ceylonensis',
'Culicicapa helianthea',
'Cephalopyrus flammiceps',
'Sylviparus modestus',
'Melanochlora sultanea',
'Periparus rufonuchalis',
'Periparus rubidiventris',
'Periparus ater',
'Pardaliparus venustulus',
'Pardaliparus elegans',
'Pardaliparus amabilis',
'Lophophanes cristatus',
'Lophophanes dichrous',
'Sittiparus varius',
'Sittiparus semilarvatus',
'Poecile superciliosus',
'Poecile lugubris',
'Poecile cinctus',
'Poecile davidi',
'Poecile hypermelaenus',
'Poecile palustris',
'Poecile weigoldicus',
'Poecile montanus',
'Cyanistes caeruleus',
'Cyanistes cyanus',
'Pseudopodoces humilis',
'Parus major',
'Parus minor',
'Parus cinereus',
'Parus monticolus',
'Machlolophus nuchalis',
'Machlolophus holsti',
'Machlolophus xanthogenys',
'Machlolophus aplonotus',
'Machlolophus spilonotus',
'Remiz pendulinus',
'Remiz macronyx',
'Remiz coronatus',
'Remiz consobrinus',
'Panurus biarmicus',
'Alaemon alaudipes',
'Ramphocoris clotbey',
'Ammomanes deserti',
'Ammomanes cinctura',
'Ammomanes phoenicura',
'Eremopterix nigriceps',
'Eremopterix griseus',
'Mirafra javanica',
'Mirafra cantillans',
'Mirafra microptera',
'Mirafra assamica',
'Mirafra erythrocephala',
'Mirafra erythroptera',
'Mirafra affinis',
'Lullula arborea',
'Alauda leucoptera',
'Alauda gulgula',
'Alauda arvensis',
'Galerida deva',
'Galerida cristata',
'Galerida malabarica',
'Eremophila alpestris',
'Eremophila bilopha',
'Calandrella acutirostris',
'Calandrella cinerea',
'Calandrella brachydactyla',
'Melanocorypha bimaculata',
'Melanocorypha calandra',
'Melanocorypha yeltoniensis',
'Melanocorypha mongolica',
'Melanocorypha maxima',
'Eremalauda dunni',
'Alaudala cheleensis',
'Alaudala rufescens',
'Alaudala raytal',
'Tricholestes criniger',
'Setornis criniger',
'Alophoixus phaeocephalus',
'Alophoixus frater',
'Alophoixus bres',
'Alophoixus flaveolus',
'Alophoixus ochraceus',
'Alophoixus pallidus',
'Alcurus striatus',
'Alcurus tympanistrigus',
'Iole finschii',
'Iole palawanensis',
'Iole viridescens',
'Iole crypta',
'Iole propinqua',
'Hemixos leucogrammicus',
'Hemixos flavala',
'Hemixos cinereus',
'Hemixos castanonotus',
'Acritillas indica',
'Ixos virescens',
'Ixos malaccensis',
'Ixos mcclellandii',
'Ixos nicobariensis',
'Hypsipetes philippinus',
'Hypsipetes mindorensis',
'Hypsipetes siquijorensis',
'Hypsipetes affinis',
'Hypsipetes longirostris',
'Hypsipetes mysticalis',
'Hypsipetes guimarasensis',
'Hypsipetes everetti',
'Hypsipetes rufigularis',
'Hypsipetes amaurotis',
'Hypsipetes thompsoni',
'Hypsipetes leucocephalus',
'Hypsipetes ganeesa',
'Euptilotus eutilotus',
'Microtarsus melanoleucos',
'Poliolophus urostictus',
'Brachypodius priocephalus',
'Brachypodius melanocephalos',
'Brachypodius fuscoflavescens',
'Brachypodius nieuwenhuisii',
'Ixodia erythropthalmos',
'Ixodia cyaniventris',
'Ixodia squamata',
'Rubigula flaviventris',
'Rubigula gularis',
'Rubigula melanicterus',
'Rubigula dispar',
'Rubigula montis',
'Nok hualon',
'Spizixos canifrons',
'Spizixos semitorques',
'Pycnonotus simplex',
'Pycnonotus plumosus',
'Pycnonotus brunneus',
'Pycnonotus zeylanicus',
'Pycnonotus cinereifrons',
'Pycnonotus luteolus',
'Pycnonotus blanfordi',
'Pycnonotus finlaysoni',
'Pycnonotus flavescens',
'Pycnonotus bimaculatus',
'Pycnonotus xantholaemus',
'Pycnonotus penicillatus',
'Pycnonotus xanthorrhous',
'Pycnonotus sinensis',
'Pycnonotus taivanus',
'Pycnonotus jocosus',
'Pycnonotus goiavier',
'Pycnonotus cafer',
'Pycnonotus aurigaster',
'Pycnonotus leucotis',
'Pycnonotus leucogenys',
'Pycnonotus xanthopygos',
'Pycnonotus barbatus',
'Pseudochelidon sirintarae',
'Neophedina cincta',
'Riparia riparia',
'Riparia diluta',
'Riparia paludicola',
'Riparia chinensis',
'Tachycineta bicolor',
'Ptyonoprogne rupestris',
'Ptyonoprogne obsoleta',
'Ptyonoprogne fuligula',
'Ptyonoprogne concolor',
'Hirundo tahitica',
'Hirundo domicola',
'Hirundo neoxena',
'Hirundo smithii',
'Hirundo rustica',
'Delichon urbicum',
'Delichon nipalense',
'Delichon dasypus',
'Cecropis daurica',
'Cecropis hyperythra',
'Cecropis striolata',
'Cecropis badia',
'Petrochelidon pyrrhonota',
'Petrochelidon fluvicola',
'Petrochelidon ariel',
'Petrochelidon nigricans',
'Pnoepyga albiventer',
'Pnoepyga formosana',
'Pnoepyga immaculata',
'Pnoepyga pusilla',
'Abroscopus superciliaris',
'Abroscopus albogularis',
'Abroscopus schisticeps',
'Phyllergates cucullatus',
'Phyllergates heterolaemus',
'Tickellia hodgsoni',
'Horornis seebohmi',
'Horornis diphone',
'Horornis carolinae',
'Horornis fortipes',
'Horornis brunnescens',
'Horornis acanthizoides',
'Horornis vulcanius',
'Horornis flavolivaceus',
'Tesia cyaniventer',
'Tesia olivea',
'Tesia everetti',
'Tesia superciliaris',
'Cettia cetti',
'Cettia major',
'Cettia brunnifrons',
'Cettia castaneocoronata',
'Urosphena squameiceps',
'Urosphena whiteheadi',
'Urosphena subulata',
'Urosphena pallidipes',
'Scotocerca inquieta',
'Aegithalos caudatus',
'Aegithalos glaucogularis',
'Aegithalos leucogenys',
'Aegithalos concinnus',
'Aegithalos niveogularis',
'Aegithalos iouschistos',
'Aegithalos bonvaloti',
'Aegithalos sharpei',
'Aegithalos fuliginosus',
'Aegithalos exilis',
'Leptopoecile sophiae',
'Leptopoecile elegans',
'Phylloscopus sibilatrix',
'Phylloscopus orientalis',
'Phylloscopus pulcher',
'Phylloscopus maculipennis',
'Phylloscopus humei',
'Phylloscopus inornatus',
'Phylloscopus subviridis',
'Phylloscopus yunnanensis',
'Phylloscopus chloronotus',
'Phylloscopus forresti',
'Phylloscopus kansuensis',
'Phylloscopus proregulus',
'Phylloscopus tytleri',
'Phylloscopus armandii',
'Phylloscopus schwarzi',
'Phylloscopus griseolus',
'Phylloscopus affinis',
'Phylloscopus fuligiventer',
'Phylloscopus fuscatus',
'Phylloscopus neglectus',
'Phylloscopus subaffinis',
'Phylloscopus trochilus',
'Phylloscopus sindianus',
'Phylloscopus collybita',
'Phylloscopus coronatus',
'Phylloscopus ijimae',
'Phylloscopus olivaceus',
'Phylloscopus cebuensis',
'Phylloscopus umbrovirens',
'Phylloscopus intermedius',
'Phylloscopus poliogenys',
'Phylloscopus burkii',
'Phylloscopus tephrocephalus',
'Phylloscopus whistleri',
'Phylloscopus valentini',
'Phylloscopus soror',
'Phylloscopus omeiensis',
'Phylloscopus nitidus',
'Phylloscopus plumbeitarsus',
'Phylloscopus trochiloides',
'Phylloscopus emeiensis',
'Phylloscopus magnirostris',
'Phylloscopus borealoides',
'Phylloscopus tenellipes',
'Phylloscopus xanthodryas',
'Phylloscopus examinandus',
'Phylloscopus borealis',
'Phylloscopus grammiceps',
'Phylloscopus montis',
'Phylloscopus calciatilis',
'Phylloscopus ricketti',
'Phylloscopus cantator',
'Phylloscopus occipitalis',
'Phylloscopus reguloides',
'Phylloscopus claudiae',
'Phylloscopus goodsoni',
'Phylloscopus ogilviegranti',
'Phylloscopus hainanus',
'Phylloscopus xanthoschistos',
'Phylloscopus trivirgatus',
'Phylloscopus nigrorum',
'Phylloscopus presbytes',
'Phylloscopus sarasinorum',
'Phylloscopus maforensis',
'Acrocephalus griseldis',
'Acrocephalus arundinaceus',
'Acrocephalus orientalis',
'Acrocephalus stentoreus',
'Acrocephalus australis',
'Acrocephalus kerearako',
'Acrocephalus bistrigiceps',
'Acrocephalus melanopogon',
'Acrocephalus paludicola',
'Acrocephalus schoenobaenus',
'Acrocephalus sorghophilus',
'Acrocephalus concinens',
'Acrocephalus tangorum',
'Acrocephalus orinus',
'Acrocephalus agricola',
'Acrocephalus dumetorum',
'Acrocephalus scirpaceus',
'Acrocephalus palustris',
'Arundinax aedon',
'Iduna caligata',
'Iduna rama',
'Iduna pallida',
'Hippolais languida',
'Hippolais olivetorum',
'Hippolais icterina',
'Robsonius rabori',
'Robsonius sorsogonensis',
'Helopsaltes fasciolatus',
'Helopsaltes amnicola',
'Helopsaltes pryeri',
'Helopsaltes certhiola',
'Helopsaltes ochotensis',
'Helopsaltes pleskei',
'Locustella lanceolata',
'Locustella fluviatilis',
'Locustella luscinioides',
'Locustella major',
'Locustella luteoventris',
'Locustella naevia',
'Locustella tacsanowskia',
'Locustella accentor',
'Locustella caudata',
'Locustella castanea',
'Locustella davidi',
'Locustella kashmirensis',
'Locustella thoracica',
'Locustella alishanensis',
'Locustella mandelli',
'Locustella montis',
'Locustella seebohmi',
'Poodytes albolimbatus',
'Poodytes gramineus',
'Malia grata',
'Cincloramphus bivittatus',
'Cincloramphus macrurus',
'Cincloramphus timoriensis',
'Megalurus palustris',
'Elaphrornis palliseri',
'Schoenicola platyurus',
'Schoenicola striatus',
'Cisticola juncidis',
'Cisticola exilis',
'Prinia crinigera',
'Prinia polychroa',
'Prinia atrogularis',
'Prinia superciliaris',
'Prinia cinereocapilla',
'Prinia buchanani',
'Prinia rufescens',
'Prinia hodgsonii',
'Prinia gracilis',
'Prinia sylvatica',
'Prinia familiaris',
'Prinia flaviventris',
'Prinia socialis',
'Prinia subflava',
'Prinia inornata',
'Micromacronus leytensis',
'Micromacronus sordidus',
'Orthotomus sutorius',
'Orthotomus atrogularis',
'Orthotomus castaneiceps',
'Orthotomus frontalis',
'Orthotomus derbianus',
'Orthotomus sericeus',
'Orthotomus ruficeps',
'Orthotomus sepium',
'Orthotomus cinereiceps',
'Orthotomus nigriceps',
'Orthotomus samarensis',
'Sylvia atricapilla',
'Sylvia borin',
'Curruca nisoria',
'Curruca minula',
'Curruca curruca',
'Curruca althaea',
'Curruca buryi',
'Curruca leucomelaena',
'Curruca crassirostris',
'Curruca nana',
'Curruca mystacea',
'Curruca ruppeli',
'Curruca melanothorax',
'Curruca melanocephala',
'Curruca cantillans',
'Curruca communis',
'Curruca conspicillata',
'Myzornis pyrrhoura',
'Moupinia poecilotis',
'Lioparus chrysotis',
'Chrysomma sinense',
'Chrysomma altirostre',
'Rhopophilus pekinensis',
'Fulvetta ruficapilla',
'Fulvetta danisi',
'Fulvetta striaticollis',
'Fulvetta vinipectus',
'Fulvetta ludlowi',
'Fulvetta manipurensis',
'Fulvetta cinereiceps',
'Fulvetta formosana',
'Calamornis heudei',
'Paradoxornis flavirostris',
'Paradoxornis guttaticollis',
'Conostoma aemodium',
'Cholornis unicolor',
'Cholornis paradoxus',
'Psittiparus gularis',
'Psittiparus margaritae',
'Psittiparus ruficeps',
'Psittiparus bakeri',
'Neosuthora davidiana',
'Suthora fulvifrons',
'Suthora nipalensis',
'Suthora verreauxi',
'Chleuasicus atrosuperciliaris',
'Sinosuthora conspicillata',
'Sinosuthora zappeyi',
'Sinosuthora brunnea',
'Sinosuthora webbiana',
'Sinosuthora alphonsiana',
'Sinosuthora przewalskii',
'Parayuhina diademata',
'Staphida everetti',
'Staphida castaniceps',
'Staphida torqueola',
'Yuhina nigrimenta',
'Yuhina brunneiceps',
'Yuhina flavicollis',
'Yuhina humilis',
'Yuhina bakeri',
'Yuhina gularis',
'Yuhina occipitalis',
'Dasycrotapha speciosa',
'Dasycrotapha pygmaea',
'Dasycrotapha plateni',
'Sterrhoptilus capitalis',
'Sterrhoptilus dennistouni',
'Sterrhoptilus nigrocapitatus',
'Zosterornis hypogrammicus',
'Zosterornis striatus',
'Zosterornis whiteheadi',
'Zosterornis latistriatus',
'Zosterornis nigrorum',
'Heleia javanica',
'Heleia pinaiae',
'Heleia squamifrons',
'Heleia goodfellowi',
'Heleia squamiceps',
'Heleia superciliaris',
'Heleia dohertyi',
'Heleia muelleri',
'Heleia crassirostris',
'Heleia wallacei',
'Apalopteron familiare',
'Tephrozosterops stalkeri',
'Zosterops ceylonensis',
'Zosterops nigrorum',
'Zosterops atricapilla',
'Zosterops abyssinicus',
'Zosterops erythropleurus',
'Zosterops emiliae',
'Zosterops japonicus',
'Zosterops palpebrosus',
'Zosterops meyeni',
'Zosterops chloris',
'Zosterops atrifrons',
'Zosterops nehrkorni',
'Zosterops consobrinorum',
'Zosterops somadikartai',
'Zosterops anomalus',
'Zosterops minor',
'Zosterops meeki',
'Zosterops atriceps',
'Zosterops buruensis',
'Zosterops stalkeri',
'Zosterops flavus',
'Zosterops citrinella',
'Zosterops everetti',
'Zosterops fuscicapilla',
'Zosterops mysorensis',
'Zosterops novaeguineae',
'Zosterops kuehni',
'Zosterops grayi',
'Zosterops uropygialis',
'Zosterops griseotinctus',
'Timalia pileata',
'Dumetia hyperythra',
'Dumetia atriceps',
'Mixornis kelleyi',
'Mixornis flavicollis',
'Mixornis gularis',
'Mixornis bornensis',
'Macronus ptilosus',
'Macronus striaticeps',
'Cyanoderma chrysaeum',
'Cyanoderma erythropterum',
'Cyanoderma melanothorax',
'Cyanoderma rufifrons',
'Cyanoderma pyrrhops',
'Cyanoderma ruficeps',
'Cyanoderma ambiguum',
'Spelaeornis caudatus',
'Spelaeornis badeigularis',
'Spelaeornis troglodytoides',
'Spelaeornis kinneari',
'Spelaeornis chocolatinus',
'Spelaeornis oatesi',
'Spelaeornis longicaudatus',
'Spelaeornis reptatus',
'Melanocichla lugubris',
'Melanocichla calva',
'Pomatorhinus ferruginosus',
'Pomatorhinus ochraceiceps',
'Pomatorhinus superciliaris',
'Pomatorhinus ruficollis',
'Pomatorhinus musicus',
'Pomatorhinus schisticeps',
'Pomatorhinus horsfieldii',
'Pomatorhinus melanurus',
'Pomatorhinus montanus',
'Erythrogenys hypoleucos',
'Erythrogenys erythrogenys',
'Erythrogenys mcclellandi',
'Erythrogenys gravivox',
'Erythrogenys swinhoei',
'Erythrogenys erythrocnemis',
'Stachyris nigricollis',
'Stachyris grammiceps',
'Stachyris maculata',
'Stachyris nigriceps',
'Stachyris poliocephala',
'Stachyris nonggangensis',
'Stachyris herberti',
'Stachyris humei',
'Stachyris roberti',
'Stachyris leucotis',
'Stachyris thoracica',
'Stachyris oglei',
'Stachyris strialata',
'Graminicola bengalensis',
'Graminicola striatus',
'Turdinus macrodactylus',
'Turdinus marmoratus',
'Turdinus atrigularis',
'Malacopteron affine',
'Malacopteron albogulare',
'Malacopteron cinereum',
'Malacopteron magnum',
'Malacopteron magnirostre',
'Malacopteron palawanense',
'Gampsorhynchus rufulus',
'Gampsorhynchus torquatus',
'Schoeniparus cinereus',
'Schoeniparus castaneceps',
'Schoeniparus klossi',
'Schoeniparus variegaticeps',
'Schoeniparus rufogularis',
'Schoeniparus dubius',
'Schoeniparus brunneus',
'Pellorneum ruficeps',
'Pellorneum fuscocapillus',
'Pellorneum palustre',
'Pellorneum capistratum',
'Pellorneum malaccense',
'Pellorneum cinereiceps',
'Pellorneum albiventre',
'Pellorneum tickelli',
'Pellorneum buettikoferi',
'Pellorneum pyrrogenys',
'Pellorneum rostratum',
'Pellorneum bicolor',
'Pellorneum celebense',
'Laticilla burnesii',
'Laticilla cinerascens',
'Kenopia striata',
'Malacocincla abbotti',
'Malacocincla sepiaria',
'Malacocincla perspicillata',
'Gypsophila crassa',
'Gypsophila brevicaudata',
'Gypsophila crispifrons',
'Gypsophila rufipectus',
'Ptilocichla mindanensis',
'Ptilocichla leucogrammica',
'Ptilocichla falcata',
'Napothera epilepidota',
'Napothera malacoptila',
'Napothera albostriata',
'Napothera pasquieri',
'Napothera naungmungensis',
'Napothera danjoui',
'Alcippe poioicephala',
'Alcippe grotei',
'Alcippe brunneicauda',
'Alcippe pyrrhoptera',
'Alcippe nipalensis',
'Alcippe davidi',
'Alcippe hueti',
'Alcippe morrisonia',
'Alcippe fratercula',
'Alcippe peracensis',
'Grammatoptila striata',
'Cutia nipalensis',
'Cutia legalleni',
'Laniellus langbianis',
'Laniellus albonotatus',
'Trochalopteron subunicolor',
'Trochalopteron austeni',
'Trochalopteron squamatum',
'Trochalopteron lineatum',
'Trochalopteron imbricatum',
'Trochalopteron virgatum',
'Trochalopteron variegatum',
'Trochalopteron affine',
'Trochalopteron morrisonianum',
'Trochalopteron henrici',
'Trochalopteron elliotii',
'Trochalopteron milnei',
'Trochalopteron yersini',
'Trochalopteron erythrocephalum',
'Trochalopteron chrysopterum',
'Trochalopteron formosum',
'Trochalopteron melanostigma',
'Trochalopteron ngoclinhense',
'Trochalopteron peninsulae',
'Montecincla cachinnans',
'Heterophasia picaoides',
'Heterophasia auricularis',
'Heterophasia capistrata',
'Heterophasia pulchella',
'Heterophasia gracilis',
'Heterophasia melanoleuca',
'Heterophasia desgodinsi',
'Actinodura nipalensis',
'Actinodura morrisoniana',
'Actinodura waldeni',
'Actinodura souliei',
'Actinodura cyanouroptera',
'Actinodura strigula',
'Actinodura egertoni',
'Actinodura ramsayi',
'Actinodura sodangorum',
'Leiothrix lutea',
'Leiothrix argentauris',
'Minla ignotincta',
'Leioptila annectens',
'Liocichla bugunorum',
'Liocichla omeiensis',
'Liocichla steerii',
'Liocichla phoenicea',
'Liocichla ripponi',
'Argya malcolmi',
'Argya cinereifrons',
'Argya longirostris',
'Argya subrufa',
'Argya rufescens',
'Argya striata',
'Argya affinis',
'Argya altirostris',
'Argya huttoni',
'Argya caudata',
'Argya squamiceps',
'Argya earlei',
'Argya gularis',
'Turdoides nipalensis',
'Garrulax merulinus',
'Garrulax annamensis',
'Garrulax canorus',
'Garrulax taewanus',
'Garrulax monileger',
'Garrulax rufifrons',
'Garrulax palliatus',
'Garrulax leucolophus',
'Garrulax bicolor',
'Garrulax milleti',
'Garrulax strepitans',
'Garrulax ferrarius',
'Garrulax maesi',
'Garrulax castanotis',
'Ianthocincla sukatschewi',
'Ianthocincla rufogularis',
'Ianthocincla konkakinhensis',
'Ianthocincla cineracea',
'Ianthocincla ocellata',
'Ianthocincla maxima',
'Ianthocincla bieti',
'Ianthocincla lunulata',
'Pterorhinus delesserti',
'Pterorhinus gularis',
'Pterorhinus vassali',
'Pterorhinus galbanus',
'Pterorhinus courtoisi',
'Pterorhinus mitratus',
'Pterorhinus treacheri',
'Pterorhinus ruficollis',
'Pterorhinus nuchalis',
'Pterorhinus chinensis',
'Pterorhinus sannio',
'Pterorhinus perspicillatus',
'Pterorhinus pectoralis',
'Pterorhinus davidi',
'Pterorhinus woodi',
'Pterorhinus lanceolatus',
'Pterorhinus waddelli',
'Pterorhinus koslowi',
'Pterorhinus albogularis',
'Pterorhinus ruficeps',
'Pterorhinus caerulatus',
'Pterorhinus berthemyi',
'Pterorhinus poecilorhynchus',
'Irena puella',
'Irena cyanogastra',
'Corthylio calendula',
'Regulus ignicapilla',
'Regulus goodfellowi',
'Regulus regulus',
'Elachura formosa',
'Troglodytes troglodytes',
'Sitta leucopsis',
'Sitta przewalskii',
'Sitta magna',
'Sitta formosa',
'Sitta azurea',
'Sitta frontalis',
'Sitta solangiae',
'Sitta oenochlamys',
'Sitta yunnanensis',
'Sitta krueperi',
'Sitta canadensis',
'Sitta villosa',
'Sitta neumayer',
'Sitta tephronota',
'Sitta arctica',
'Sitta victoriae',
'Sitta himalayensis',
'Sitta europaea',
'Sitta nagaensis',
'Sitta cashmirensis',
'Sitta castanea',
'Sitta cinnamoventris',
'Sitta neglecta',
'Tichodroma muraria',
'Certhia familiaris',
'Certhia hodgsoni',
'Certhia brachydactyla',
'Certhia himalayana',
'Certhia nipalensis',
'Certhia discolor',
'Certhia manipurensis',
'Certhia tianquanensis',
'Salpornis spilonota',
'Aplonis metallica',
'Aplonis circumscripta',
'Aplonis mystacea',
'Aplonis cantoroides',
'Aplonis crassa',
'Aplonis magna',
'Aplonis panayensis',
'Aplonis mysolensis',
'Aplonis minor',
'Mino dumontii',
'Mino anais',
'Basilornis celebensis',
'Basilornis galeatus',
'Basilornis corythaix',
'Goodfellowia miranda',
'Sarcops calvus',
'Streptocitta albicollis',
'Streptocitta albertinae',
'Enodes erythrophris',
'Scissirostrum dubium',
'Saroglossa spilopterus',
'Ampeliceps coronatus',
'Gracula ptilogenys',
'Gracula religiosa',
'Gracula indica',
'Gracula robusta',
'Gracula enganensis',
'Acridotheres grandis',
'Acridotheres cristatellus',
'Acridotheres javanicus',
'Acridotheres cinereus',
'Acridotheres fuscus',
'Acridotheres albocinctus',
'Acridotheres ginginianus',
'Acridotheres tristis',
'Acridotheres melanopterus',
'Acridotheres burmannicus',
'Spodiopsar sericeus',
'Spodiopsar cineraceus',
'Gracupica nigricollis',
'Gracupica contra',
'Agropsar sturninus',
'Agropsar philippensis',
'Sturnia sinensis',
'Sturnia malabarica',
'Sturnia erythropygia',
'Sturnia blythii',
'Sturnia pagodarum',
'Sturnornis albofrontatus',
'Leucopsar rothschildi',
'Pastor roseus',
'Sturnus vulgaris',
'Creatophora cinerea',
'Onychognathus tristramii',
'Rhabdornis mystacalis',
'Rhabdornis inornatus',
'Rhabdornis grandis',
'Grandala coelicolor',
'Cataponera turdoides',
'Chlamydochaera jeffe',
]

In [15]:
len(All_birds)

1423

In [16]:
len(birds)

999

In [17]:
Final_birds = []
for i in All_birds:
    if(i not in birds):
        Final_birds.append(i)
Final_birds

['Loriculus camiguinensis',
 'Loriculus galgulus',
 'Loriculus stigmatus',
 'Loriculus amabilis',
 'Loriculus sclateri',
 'Loriculus catamene',
 'Loriculus aurantiifrons',
 'Loriculus exilis',
 'Loriculus pusillus',
 'Loriculus flosculus',
 'Psarisomus dalhousiae',
 'Corydon sumatranus',
 'Sarcophanops samarensis',
 'Sarcophanops steerii',
 'Serilophus lunatus',
 'Cymbirhynchus macrorhynchos',
 'Eurylaimus javanicus',
 'Eurylaimus ochromalus',
 'Calyptomena viridis',
 'Calyptomena hosii',
 'Calyptomena whiteheadi',
 'Hydrornis phayrei',
 'Hydrornis caeruleus',
 'Hydrornis oatesi',
 'Hydrornis schneideri',
 'Hydrornis nipalensis',
 'Hydrornis soror',
 'Hydrornis irena',
 'Hydrornis guajanus',
 'Hydrornis schwaneri',
 'Hydrornis baudii',
 'Hydrornis cyaneus',
 'Hydrornis gurneyi',
 'Hydrornis elliotii',
 'Erythropitta kochi',
 'Erythropitta erythrogaster',
 'Erythropitta venusta',
 'Erythropitta granatina',
 'Erythropitta ussheri',
 'Erythropitta arquata',
 'Pitta brachyura',
 'Pitta mol

In [18]:
len(Final_birds)

1423

In [19]:
Final_birds[0]

'Loriculus camiguinensis'

In [20]:
3002-999

2003

In [21]:
# Final_birds = []
for i in Final_birds:
    if i in birds:
        print(i)
#         Final_birds.append(i)
#Final_birds

In [22]:
len(Final_birds)

1423

In [23]:
Final_birds

['Loriculus camiguinensis',
 'Loriculus galgulus',
 'Loriculus stigmatus',
 'Loriculus amabilis',
 'Loriculus sclateri',
 'Loriculus catamene',
 'Loriculus aurantiifrons',
 'Loriculus exilis',
 'Loriculus pusillus',
 'Loriculus flosculus',
 'Psarisomus dalhousiae',
 'Corydon sumatranus',
 'Sarcophanops samarensis',
 'Sarcophanops steerii',
 'Serilophus lunatus',
 'Cymbirhynchus macrorhynchos',
 'Eurylaimus javanicus',
 'Eurylaimus ochromalus',
 'Calyptomena viridis',
 'Calyptomena hosii',
 'Calyptomena whiteheadi',
 'Hydrornis phayrei',
 'Hydrornis caeruleus',
 'Hydrornis oatesi',
 'Hydrornis schneideri',
 'Hydrornis nipalensis',
 'Hydrornis soror',
 'Hydrornis irena',
 'Hydrornis guajanus',
 'Hydrornis schwaneri',
 'Hydrornis baudii',
 'Hydrornis cyaneus',
 'Hydrornis gurneyi',
 'Hydrornis elliotii',
 'Erythropitta kochi',
 'Erythropitta erythrogaster',
 'Erythropitta venusta',
 'Erythropitta granatina',
 'Erythropitta ussheri',
 'Erythropitta arquata',
 'Pitta brachyura',
 'Pitta mol